# Review Rec Bot

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loguru import logger
import json

import sys
sys.path.insert(0, '..')

In [3]:
import os

from tqdm.notebook import tqdm
import pandas as pd
from llama_index.core.response.notebook_utils import display_source_node

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

# Arguments

In [6]:
from src.run.args import RunInputArgs

ARGS = RunInputArgs(
    EXPERIMENT_NAME="Review Rec Bot - Yelp Review Rec Bot",
    RUN_NAME="025_modify_prompt_with_tree_summarize",
    RUN_DESCRIPTION="""
# Objective
Make the response from the LLM more adapted to the task of recommending places.

# Implementation
- ResponseMode=TreeSummarize
- Customize prompt

## Changelog
""",
    TESTING=False,
    LOG_TO_MLFLOW=True,
    OBSERVABILITY=True,
    RECREATE_INDEX=False,
    RECREATE_RETRIEVAL_EVAL_DATASET=False,
    RECREATE_RESPONSE_EVAL_DATASET=False,
    DEBUG=False,
)

ARGS

{
  "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
  "RUN_NAME": "025_modify_prompt_with_tree_summarize",
  "RUN_DESCRIPTION": "\n# Objective\nMake the response from the LLM more adapted to the task of recommending places.\n\n# Implementation\n- ResponseMode=TreeSummarize\n- Customize prompt\n\n## Changelog\n",
  "TESTING": false,
  "DEBUG": false,
  "OBSERVABILITY": true,
  "LOG_TO_MLFLOW": true,
  "RECREATE_RETRIEVAL_EVAL_DATASET": false,
  "RECREATE_RESPONSE_EVAL_DATASET": false,
  "RECREATE_INDEX": false
}

# Load config

In [7]:
from src.run.cfg import RunConfig

In [8]:
cfg = RunConfig()
cfg.init(ARGS)

2024-08-11 09:26:04.424 | INFO     | src.run.cfg:init:148 - Starting Observability server with Phoenix...
INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/dvquys/.phoenix
2024-08-11 09:26:08.887 | INFO     | src.run.cfg:init:166 - Setting up MLflow experiment Review Rec Bot - Yelp Review Rec Bot - run 025_modify_prompt_with_tree_summarize...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


2024-08-11 09:26:09.104 | INFO     | src.run.cfg:init:175 - Notebook-generated artifacts are persisted at data/025_modify_prompt_with_tree_summarize


In [9]:
cfg

/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


{
  "args": {
    "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
    "RUN_NAME": "025_modify_prompt_with_tree_summarize",
    "RUN_DESCRIPTION": "\n# Objective\nMake the response from the LLM more adapted to the task of recommending places.\n\n# Implementation\n- ResponseMode=TreeSummarize\n- Customize prompt\n\n## Changelog\n",
    "TESTING": false,
    "DEBUG": false,
    "OBSERVABILITY": true,
    "LOG_TO_MLFLOW": true,
    "RECREATE_RETRIEVAL_EVAL_DATASET": false,
    "RECREATE_RESPONSE_EVAL_DATASET": false,
    "RECREATE_INDEX": false
  },
  "app_name": "review_rec_bot",
  "storage_context_persist_dp": "data/018_finetuned_embedding_reindex/storage_context",
  "db_collection": "review_rec_bot__018_finetuned_embedding_reindex__huggingface____data_finetune_embedding_finetuned_model",
  "notebook_cache_dp": "data/025_modify_prompt_with_tree_summarize",
  "data_fp": "../data/yelp_dataset/sample/sample_100_biz/denom_review.parquet",
  "llm_cfg": {
    "llm_provider": "openai

## Set up logger to collect additional info

In [10]:
collect_fp = f"{cfg.notebook_cache_dp}/collect.log"
logger.add(collect_fp, filter=lambda record: "[COLLECT]" in record['message'], mode='w')

1

# Load input data

In [11]:
data = pd.read_parquet(cfg.data_fp)
data = data.assign(
    biz_categories=lambda df: df['biz_categories'].str.split(', '),
    date=lambda df: df['date'].dt.strftime('%Y-%m-%dT%H:%M:%S')
)
logger.info(f"[COLLECT] {len(data)=}")
data.iloc[0]

2024-08-11 09:26:09.266 | INFO     | __main__:<module>:6 - [COLLECT] len(data)=5240


review_id                                      L0jv8c2FbpWSlfNC6bbUEA
user_id                                        bFPdtzu11Oi0f92EAcjqmg
business_id                                    IDtLPgUrqorrpqSLdfMhZQ
review_stars                                                        5
useful                                                              0
funny                                                               0
cool                                                                0
text                What a great addition to the Funk Zone!  Grab ...
date                                              2016-10-13T22:50:47
biz_name                                         Helena Avenue Bakery
biz_address                                     131 Anacapa St, Ste C
biz_city                                                Santa Barbara
biz_state                                                          CA
biz_postal_code                                                 93101
biz_latitude        

In [12]:
input_data = data
if ARGS.TESTING:
    input_data = data[:20]
logger.info(f"[COLLECT] {len(input_data)=}")

2024-08-11 09:26:09.295 | INFO     | __main__:<module>:4 - [COLLECT] len(input_data)=5240


In [13]:
input_data.columns

Index(['review_id', 'user_id', 'business_id', 'review_stars', 'useful',
       'funny', 'cool', 'text', 'date', 'biz_name', 'biz_address', 'biz_city',
       'biz_state', 'biz_postal_code', 'biz_latitude', 'biz_longitude',
       'biz_stars', 'biz_review_count', 'biz_is_open', 'biz_attributes',
       'biz_categories', 'biz_hours'],
      dtype='object')

# Prepare documents

In [14]:
from llama_index.core import Document

documents = []
embedding_visible_metadata = ["review_stars", "biz_name", "biz_address", "biz_city", "biz_state", "biz_categories"]
excluded_embed_metadata_keys = [k for k in input_data.columns if k not in embedding_visible_metadata]

for i, row in tqdm(input_data.iterrows(), total=len(input_data)):
    record = row.to_dict()
    text = record['text']
    metadata = {k: v for k, v in record.items() if k not in ('text')}
    
    doc = Document(
        text=text,
        metadata=metadata,
        excluded_embed_metadata_keys=excluded_embed_metadata_keys,
        excluded_llm_metadata_keys=excluded_embed_metadata_keys
    )
    documents.append(doc)

logger.info(f"[COLLECT] {len(documents)=}")

  0%|          | 0/5240 [00:00<?, ?it/s]

2024-08-11 09:26:09.849 | INFO     | __main__:<module>:20 - [COLLECT] len(documents)=5240


#### Check document embedding text

In [15]:
from llama_index.core.schema import MetadataMode

In [16]:
document = documents[0]
print(document.get_content(metadata_mode=MetadataMode.EMBED))

review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries']

What a great addition to the Funk Zone!  Grab a bite, grab some tastings, life is good. Right next door to the Santa Barbara Wine Collective, in fact it actually shares the same tables.  We had a fabulous savory croissant.


# Set up LLM

In [17]:
llm, embed_model = cfg.setup_llm()

WARNI [sentence_transformers.SentenceTransformer] You try to use a model that was created with version 2.7.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [18]:
print(cfg.llm_cfg.model_dump_json(indent=2))

{
  "llm_provider": "openai",
  "llm_model_name": "gpt-4o-mini",
  "embedding_provider": "huggingface",
  "embedding_model_name": "./data/finetune_embedding/finetuned_model",
  "embedding_model_dim": 768,
  "ollama__host": "192.168.100.14",
  "ollama__port": 11434
}


In [19]:
from llama_index.core import Settings
Settings.embed_model = embed_model
Settings.llm = llm

# Vector Store

In [20]:
import qdrant_client
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

from src.run.orchestrator import RunOrchestrator

In [21]:
qdrantdb = qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)
aqdrantdb = qdrant_client.AsyncQdrantClient(
    host="localhost",
    port=6333
)

RunOrchestrator.setup_db(cfg, qdrantdb)

db_collection = qdrantdb.get_collection(cfg.db_collection)
vector_store = QdrantVectorStore(
    client=qdrantdb,
    collection_name=cfg.db_collection,
    aclient=aqdrantdb,
    enable_hybrid=False,
    prefer_grpc=True
)

2024-08-11 09:26:13.574 | INFO     | src.run.orchestrator:setup_db:37 - Use existing Qdrant collection: review_rec_bot__018_finetuned_embedding_reindex__huggingface____data_finetune_embedding_finetuned_model
WARNI [llama_index.vector_stores.qdrant.base] Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.


# Index Embeddings

In [22]:
import time
import pickle
from multiprocessing import set_start_method
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.node_parser import SemanticSplitterNodeParser
# from llama_index.core.node_parser import SentenceSplitter

In [23]:
chunker = SemanticSplitterNodeParser
chunker_cfg = {
    "buffer_size": 1,
    "breakpoint_percentile_threshold": 95,
    "embed_model": embed_model
}

# chunker = SentenceSplitter
# chunker_cfg = {
#     "chunk_size": 512,
#     "chunk_overlap": 10
# }

In [24]:
t0 = time.perf_counter()
# TODO: TO understand the differences between points_count and indexed_vector_counts.
# Here indexed_vector_counts = 0
db_collection_count = db_collection.points_count

if db_collection_count > 0 and ARGS.RECREATE_INDEX == False:
    logger.info(f"Loading Storage Context from {cfg.storage_context_persist_dp}...")
    docstore = SimpleDocumentStore.from_persist_dir(persist_dir=cfg.storage_context_persist_dp)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
    nodes = list(docstore.docs.values())
else:
    logger.info(f"Creating new DB index...")
    from llama_index.core.extractors import TitleExtractor
    from llama_index.core.ingestion import IngestionPipeline, IngestionCache

    # create the pipeline with transformations
    pipeline = IngestionPipeline(
        transformations=[
            chunker(**chunker_cfg),
            embed_model,
        ],
        vector_store = vector_store
    )

    num_workers = None
    # TODO: I can get the preprocessing completed much quicker if I push the initial steps through a multiprocessing loop and then separately create the embeddings using the built-in batching already provided in the SentenceTransformer encode method.
    # Ref: https://github.com/run-llama/llama_index/issues/10104#issuecomment-1899401584
    # Currently setting num_workers leads to code simple hang
    # Ref: https://github.com/run-llama/llama_index/issues/10104
    # num_workers = os.cpu_count() - 1
    # os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    # set_start_method("spawn", force=True)  # it hangs without this line
    logger.info(f"Running Ingestion Pipeline with {num_workers=}...")
    nodes = await pipeline.arun(documents=documents, num_workers=num_workers, show_progress=True)

    # Define Docstore as an abstraction on top of nodes to easily manage (e.g. get node by id)
    docstore = SimpleDocumentStore()
    await docstore.async_add_documents(nodes)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
    logger.info(f"Persisting Storage Context to {cfg.storage_context_persist_dp}...")
    storage_context.persist(cfg.storage_context_persist_dp)

t1 = time.perf_counter()

2024-08-11 09:26:13.697 | INFO     | __main__:<module>:7 - Loading Storage Context from data/018_finetuned_embedding_reindex/storage_context...


In [25]:
logger.info(f"Indexing {len(documents)} documents into VectorStoreIndex took {t1 - t0:,.0f}s")
logger.info(f"[COLLECT] {len(nodes)=}")

2024-08-11 09:26:21.383 | INFO     | __main__:<module>:1 - Indexing 5240 documents into VectorStoreIndex took 8s
2024-08-11 09:26:21.384 | INFO     | __main__:<module>:2 - [COLLECT] len(nodes)=10219


# Analyze Chunks

In [26]:
for i, node in enumerate(nodes[:5]):
    print(f"\n\n==========Node {i+1}============")
    print(node.metadata)
    print(node.get_text())



==========Node 1============
{'review_id': 'L0jv8c2FbpWSlfNC6bbUEA', 'user_id': 'bFPdtzu11Oi0f92EAcjqmg', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-10-13T22:50:47', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly':

# Construct Retriever

In [27]:
from llama_index.core.retrievers import VectorIndexRetriever

In [28]:
print(cfg.retrieval_cfg.model_dump_json(indent=2))

{
  "retrieval_top_k": 50,
  "retrieval_dense_top_k": 50,
  "retrieval_sparse_top_k": 50,
  "retrieval_similarity_cutoff": null,
  "rerank_top_k": 10,
  "rerank_model_name": "BAAI/bge-reranker-v2-m3"
}


In [29]:
# configure retriever
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)
vector_retriever = VectorIndexRetriever(
    index=index,
    vector_store_query_mode='mmr',
    similarity_top_k=cfg.retrieval_cfg.retrieval_dense_top_k,
    # sparse_top_k=cfg.retrieval_cfg.retrieval_sparse_top_k,
)

In [30]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=cfg.retrieval_cfg.retrieval_sparse_top_k,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

bm25_retrieval_results = bm25_retriever.retrieve("Nashville, grocery")
for result in bm25_retrieval_results:
    display_source_node(result, source_length=1000, show_source_metadata=True)

DEBUG [bm25s] Building index from IDs objects


**Node ID:** 85d45fd0-53db-4bac-82b0-f7eceea27be5<br>**Similarity:** 3.2125637531280518<br>**Text:** Convenient grocery store. A bit ghetto, and not because of it's location at all.<br>**Metadata:** {'review_id': '-kdFaer2K56NJCGNadTe5w', 'user_id': 'UYgAC9mvMYc12Ufc5KVUnA', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 3, 'useful': 1, 'funny': 1, 'cool': 0, 'date': '2010-10-27T15:07:05', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': None, 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': None, 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': None, 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': None, 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': None, 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': None, 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Food', 'Grocery'], 'biz_hours': {'Friday': '6:0-1:0', 'Monday': '6:0-1:0', 'Saturday': '6:0-1:0', 'Sunday': '6:0-1:0', 'Thursday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'Wednesday': '6:0-1:0'}}<br>

**Node ID:** 4d1a9d64-f9da-4eef-af8e-00be4b2b2a56<br>**Similarity:** 3.1722183227539062<br>**Text:** A good grocery store, just requires a car to get to (or bus). This used to be somewhere they shuttled local university students from Penn to come get groceries; not sure if they still do that.

There's almost always a good deal to be had, fresh produce and meat, and feels like a truly suburban grocery store. Lines are long at times, but the only real problem is the distance from the city if you don't have a car.<br>**Metadata:** {'review_id': 'hKTTE9mOS1tXI6gDbDU_Mw', 'user_id': 'nnwBdqGHIAJQ5QX9lHOtrQ', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 3, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2009-01-20T16:11:51', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': None, 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': None, 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': None, 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': None, 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': None, 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': None, 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Food', 'Grocery'], 'biz_hours': {'Friday': '6:0-1:0', 'Monday': '6:0-1:0', 'Saturday': '6:0-1:0', 'Sunday': '6:0-1:0', 'Thursday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'Wednesday': '6:0-1:0'}}<br>

**Node ID:** ea019f4c-6507-4f1b-a59e-87ecfa981cc6<br>**Similarity:** 3.0762240886688232<br>**Text:** This grocery store SUCKS!  But it's the closest one to my house, so I go there for things that I need fast.  The produce is the worst possible.  I don't think the prices are very good, but then again it seems that all grocery stores in Philly are outrageously priced.  Employees are the slowest moving in the world!<br>**Metadata:** {'review_id': 'krw5FlAWXS16NkkqViK-wg', 'user_id': 'B-d6QdtkkoLHCyIyuT6y4A', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 2, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2011-05-12T20:29:04', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': None, 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': None, 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': None, 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': None, 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': None, 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': None, 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Food', 'Grocery'], 'biz_hours': {'Friday': '6:0-1:0', 'Monday': '6:0-1:0', 'Saturday': '6:0-1:0', 'Sunday': '6:0-1:0', 'Thursday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'Wednesday': '6:0-1:0'}}<br>

**Node ID:** 75c476c7-332e-40c1-8533-903afc4bf975<br>**Similarity:** 3.0317909717559814<br>**Text:** Free parking garage.  Eat in grocery store or deli type restaurant....everything is by the pound.<br>**Metadata:** {'review_id': 'vQxUaNanrYGZ_X3qCiIC3g', 'user_id': 'btAiOvMIfAbP5Sc5fAR8zA', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2017-07-08T18:07:49', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'beer_and_wine'", 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': True, 'casual': True}", 'BYOB': None, 'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}", 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': True, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'HappyHour': 'True', 'HasTV': 'False', 'Music': "{'dj': False, 'background_music': False, 'no_music': False, 'jukebox': False, 'live': False, 'video': False, 'karaoke': False}", 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': "u'no'", 'WheelchairAccessible': 'True', 'WiFi': "'free'"}, 'biz_categories': ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners'], 'biz_hours': {'Friday': '7:0-17:0', 'Monday': '7:0-17:0', 'Saturday': '7:0-17:0', 'Sunday': '7:0-17:0', 'Thursday': '7:0-17:0', 'Tuesday': '7:0-17:0', 'Wednesday': '7:0-17:0'}}<br>

...

**Node ID:** e2ad18af-9ce8-4d8c-ad8d-34c8f91f55ab<br>**Similarity:** 2.1714556217193604<br>**Text:** Handcrafted ice cream right in the heart of Nashville. The place is always busy but worth the wait.<br>**Metadata:** {'review_id': '073JO4AkBwFz-6VoFSB--A', 'user_id': 'wa1PJGn1XxhIFbGWTeIbOQ', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2019-07-07T22:57:17', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': 'False', 'Caters': 'False', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': 'False', 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': None, 'HappyHour': None, 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'None', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': None, 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food'], 'biz_hours': {'Friday': '8:0-0:0', 'Monday': '8:0-23:0', 'Saturday': '8:0-0:30', 'Sunday': '8:0-23:0', 'Thursday': '8:0-23:0', 'Tuesday': '8:0-23:0', 'Wednesday': '8:0-23:0'}}<br>

**Node ID:** 021cdfe8-157e-401f-a1d3-25df7836fdc5<br>**Similarity:** 2.1714556217193604<br>**Text:** I had the Nashville Hot Chicken Salad and it was wonderful! Big enough to eat for lunch and dinner!<br>**Metadata:** {'review_id': 'R2AI3QGNuWNP2sHPd40ewg', 'user_id': '1xY7RkCcAzS-KBhZ0ZEE0w', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2019-10-23T18:24:14', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'beer_and_wine'", 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': True, 'casual': True}", 'BYOB': None, 'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}", 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': True, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'HappyHour': 'True', 'HasTV': 'False', 'Music': "{'dj': False, 'background_music': False, 'no_music': False, 'jukebox': False, 'live': False, 'video': False, 'karaoke': False}", 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': "u'no'", 'WheelchairAccessible': 'True', 'WiFi': "'free'"}, 'biz_categories': ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners'], 'biz_hours': {'Friday': '7:0-17:0', 'Monday': '7:0-17:0', 'Saturday': '7:0-17:0', 'Sunday': '7:0-17:0', 'Thursday': '7:0-17:0', 'Tuesday': '7:0-17:0', 'Wednesday': '7:0-17:0'}}<br>

**Node ID:** eb54e826-47cf-42f4-bdfd-03129de213e0<br>**Similarity:** 2.162130832672119<br>**Text:** the cocktails were great & we ordered plenty of hot sake to enjoy our evening. the main reason why I am giving it 3 stars is because the menu is a little pricey (location is downtown nashville off 3rd) & nothing spectacular stood out for it being a new spot in nashville.<br>**Metadata:** {'review_id': 'OI6hq_7xWuK4JuFpkw_nqw', 'user_id': 'lU_H3xHRiWFkVStKX9sd9g', 'business_id': 'tMkwHmWFUEXrC9ZduonpTg', 'review_stars': 3, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-12-16T01:36:24', 'biz_name': 'The Green Pheasant', 'biz_address': '215 1st Ave S', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1598858, 'biz_longitude': -86.7731974, 'biz_stars': 4.0, 'biz_review_count': 161, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'full_bar'", 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': True, 'casual': False}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'False', 'BusinessAcceptsBitcoin': 'False', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': True, 'street': False, 'validated': True, 'lot': False, 'valet': True}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'False', 'GoodForMeal': "{'dessert': None, 'latenight': None, 'lunch': None, 'dinner': True, 'brunch': None, 'breakfast': None}", 'HappyHour': 'True', 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'False', 'RestaurantsAttire': "u'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': None, 'RestaurantsReservations': 'True', 'RestaurantsTableService': 'True', 'RestaurantsTakeOut': 'None', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'free'"}, 'biz_categories': ['Restaurants', 'Japanese', 'Seafood'], 'biz_hours': {'Friday': '16:0-23:0', 'Monday': None, 'Saturday': '16:0-23:0', 'Sunday': None, 'Thursday': '16:0-22:0', 'Tuesday': None, 'Wednesday': '16:0-22:0'}}<br>

**Node ID:** 7556504b-ce3a-47f6-8cde-956fa07fbecc<br>**Similarity:** 2.1461713314056396<br>**Text:** Stopped here on a whim for lunch on our way home to Orlando after a weekend at the beach. Delicious sandwiches, lasagna, ziti and salad. Bought some wonderful Italian groceries including fresh mozzarella  and fresh bread to take home. Very friendly place...family business.<br>**Metadata:** {'review_id': 'LC8mOm9RulAdyNlI9a2GNQ', 'user_id': 'UwVqHjSqTRz3FUgr2TPsKg', 'business_id': '0bPLkL0QhhPO5kt1_EXmNQ', 'review_stars': 5, 'useful': 1, 'funny': 0, 'cool': 0, 'date': '2015-12-30T20:36:26', 'biz_name': "Zio's Italian Market", 'biz_address': '2575 E Bay Dr', 'biz_city': 'Largo', 'biz_state': 'FL', 'biz_postal_code': '33771', 'biz_latitude': 27.9161159, 'biz_longitude': -82.7604608, 'biz_stars': 4.5, 'biz_review_count': 100, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': 'False', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': False, 'breakfast': False}", 'HappyHour': None, 'HasTV': 'True', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'False', 'RestaurantsAttire': "u'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'False', 'RestaurantsPriceRange2': '1', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'no'"}, 'biz_categories': ['Food', 'Delis', 'Italian', 'Bakeries', 'Restaurants'], 'biz_hours': {'Friday': '10:0-20:0', 'Monday': '10:0-18:0', 'Saturday': '10:0-20:0', 'Sunday': None, 'Thursday': '10:0-20:0', 'Tuesday': '10:0-20:0', 'Wednesday': '10:0-20:0'}}<br>

**Node ID:** 5d3cd77a-fcd9-4ef4-ba65-0c4ba5c5a220<br>**Similarity:** 2.142127275466919<br>**Text:** After visiting Mike's Ice Cream in Nashville, I am now saddened that my favorite spot is across the country. The home made Vanilla instantly reminded me of my youth when my dad used to make ice cream in the summer. If you are in Nashville, I highly recommend Mike's.<br>**Metadata:** {'review_id': 'rtUBWJgwUptv1fvulsGmnQ', 'user_id': 'E1Gfpm8X0JB-x0E7Gsa2qg', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2012-06-01T21:50:16', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': 'False', 'Caters': 'False', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': 'False', 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': None, 'HappyHour': None, 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'None', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': None, 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food'], 'biz_hours': {'Friday': '8:0-0:0', 'Monday': '8:0-23:0', 'Saturday': '8:0-0:30', 'Sunday': '8:0-23:0', 'Thursday': '8:0-23:0', 'Tuesday': '8:0-23:0', 'Wednesday': '8:0-23:0'}}<br>

In [31]:
# test_query = nodes[3].text
test_query = "Received confirmation of my order at 5 pm"
vector_results = vector_retriever.retrieve(test_query)
bm25_results = bm25_retriever.retrieve(test_query)

# TODO: Reconsider this test.
# Currently it aims to make sure the vector retriever, 
# when given a piece of text extracted from a node, 
# would return the original node.
# But what is the value of it?

# Check if the original node in top 10:
found = False
for vector_result in vector_results[:10]:
    target_id = bm25_results[0].id_
    if vector_result.id_ == target_id:
        found = True
        break
# assert found, "Can not find the original node in the top 10 vector results"
# assert bm25_results[0].id_ == vector_results[0].id_, "Different vector_retriever nodes vs bm25_retriever nodes"

In [32]:
from llama_index.core.retrievers import QueryFusionRetriever

# query_gen_prompt = """
# You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
# You should apply different techniques to create new strings. Here are some example techniques:
# - Technique 1 - Optimize for full-text search: Extract the key concepts and entities as keywords. This technique can generate multiple strings instead of just one. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  string 1 = "cold brew coffee", string 2 = "hanoi". Do not modify the keyword, keep the keyword exactly in the input query. Return each output string on a new line.
# - Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."
# - Technique 3 - Increase recall: Paraphase the original question into a new question but use different synonyms or broader concepts. Example input query: "How to find some good local grocery stores near Changi Singapore?" -> Expected output query: "Where to find a good mart selling Singaporean stuffs to the east of Singapore?"

# Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2, third technique 3. If run of of techniques then re-iterate from the start.

# Return one string on each line, related to the input query.

# Only return the strings. Never include the chosen technique.

# Input Query: {query}\n
# New strings:\n
# """

# query_gen_prompt = """
# You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
# You should apply different techniques to create new strings. Here are some example techniques:
# - Technique 1 - Optimize for full-text search: Rephrase the input query to contain only important keywords. Remove all stopwords and low information words. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  "cold brew coffee hanoi"
# - Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."
# - Technique 3 - Increase recall: Paraphase the original question into a new question but use different synonyms or broader concepts. Example input query: "How to find some good local grocery stores near Changi Singapore?" -> Expected output query: "Where to find a good mart selling Singaporean stuffs to the east of Singapore?"

# Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2, third technique 3. If run of of techniques then re-iterate from the start.

# Return one string on each line, related to the input query.

# Only return the strings. Never include the chosen technique.

# Input Query: {query}\n
# New strings:\n
# """

query_gen_prompt = """
You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
You should apply different techniques to create new strings. Here are some example techniques:
- Technique 1 - Optimize for full-text search: Rephrase the input query to contain only important keywords. Remove all stopwords and low information words. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  "cold brew coffee hanoi"
- Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."

Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2. If run of of techniques then re-iterate from the start.

Return one string on each line, related to the input query.

Only return the strings. Never include the chosen technique.

Input Query: {query}\n
New strings:\n
"""

from llama_index.llms.openai import OpenAI
llm = OpenAI(model=cfg.eval_cfg.response_eval_llm_model, **cfg.eval_cfg.response_eval_llm_model_config)

retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    llm=llm,
    similarity_top_k=cfg.retrieval_cfg.retrieval_top_k,
    num_queries=2,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    query_gen_prompt=query_gen_prompt
)

## Test retrieval

In [33]:
query = "What are some places to enjoy cold brew coffee"
retrieval_results = await retriever.aretrieve(query)
for node in retrieval_results:
    display_source_node(node, source_length=1000)
    print(node.get_content(metadata_mode=MetadataMode.LLM))
    print("\n")

Generated queries:
cold brew coffee places
"I recently visited a cozy café that specializes in cold brew coffee, and I was impressed by their unique flavors and relaxing atmosphere. It's a perfect spot to unwind and enjoy a refreshing drink."


**Node ID:** 64f06176-a59e-44be-b332-4507fc783195<br>**Similarity:** 0.06191025457811894<br>**Text:** If it's cold, come grab a coffee instead. They offer all the great styles and flavors of coffee as any other coffee house you'll find.<br>

review_stars: 5
biz_name: Mike's Ice Cream
biz_address: 129 2nd Ave N
biz_city: Nashville
biz_state: TN
biz_categories: ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food']

If it's cold, come grab a coffee instead. They offer all the great styles and flavors of coffee as any other coffee house you'll find.




**Node ID:** 4e272c53-d4a4-4e15-96f1-efa6c8d09e00<br>**Similarity:** 0.06187941713888418<br>**Text:** I'm a fan of dark roast but it's usually not available. My advice brew different flavors of coffee and keep it hot.<br>

review_stars: 4
biz_name: Wawa
biz_address: 3604 Chestnut St
biz_city: Philadelphia
biz_state: PA
biz_categories: ['Restaurants', 'Automotive', 'Delis', 'Gas Stations', 'Food', 'Coffee & Tea', 'Sandwiches', 'Convenience Stores']

I'm a fan of dark roast but it's usually not available. My advice brew different flavors of coffee and keep it hot.




...

review_stars: 4
biz_name: Bier Brewery and Tap Room
biz_address: 5133 E 65th St
biz_city: Indianapolis
biz_state: IN
biz_categories: ['Food', 'Beer', 'Wine & Spirits', 'Breweries']

After enjoying their current lineup, I walked around the relaxed tap room, decked out with couches and lounging chairs. Oh, and if you want further proof that Bier is about quality in addition to quantity, make sure you check out their wall of fame, covered in numerous awards and ribbons from various brewing competitions, including the Great American Beer Festival. 

Just like one of their self-proclaimed/named brews, PDG, this place is pretty damn good. 'Nuff said. 4.5 Stars.




**Node ID:** 3c5845ac-597d-4410-8d53-601ecc5fe313<br>**Similarity:** 0.021869488536155203<br>**Text:** By far, my favourite coffeehouse in the city! I have never had great coffee until I started going to Roast. 

We stop in at least once a week, more if I have the time. I usually have the pleasure of seeing Josh in the mornings, and he is just so friendly (as is all the staff there) and I am pretty sure he remembered my name after the first time I came to Roast.

The drinks are all fabulous, and the small amount of food I have tried here, I have enjoyed.<br>

review_stars: 5
biz_name: Roast Coffeehouse and Wine Bar
biz_address: 10359 104 Street NW
biz_city: Edmonton
biz_state: AB
biz_categories: ['Coffee & Tea', 'Food', 'Cafes', 'Bars', 'Wine Bars', 'Restaurants', 'Nightlife']

By far, my favourite coffeehouse in the city! I have never had great coffee until I started going to Roast. 

We stop in at least once a week, more if I have the time. I usually have the pleasure of seeing Josh in the mornings, and he is just so friendly (as is all the staff there) and I am pretty sure he remembered my name after the first time I came to Roast.

The drinks are all fabulous, and the small amount of food I have tried here, I have enjoyed.




**Node ID:** d6b7438c-6ae3-4c89-9956-53a9fc65fc9e<br>**Similarity:** 0.02116935483870968<br>**Text:** My friends and I were about to start our pub crawl but first things first a couple of us wanted coffee. Someone pointed out Mike's Ice Cream which had coffee.<br>

review_stars: 4
biz_name: Mike's Ice Cream
biz_address: 129 2nd Ave N
biz_city: Nashville
biz_state: TN
biz_categories: ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food']

My friends and I were about to start our pub crawl but first things first a couple of us wanted coffee. Someone pointed out Mike's Ice Cream which had coffee.




## Retrieval Evaluation

### Synthetic

In [34]:
from src.run.eval import RetrievalEvaluator

In [35]:
retrieval_evaluator = RetrievalEvaluator()
retrieval_evaluator.generate_synthetic_dataset(cfg, nodes)

2024-08-11 09:26:29.160 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:79 - Loading retrieval_eval_nodes from data/018_finetuned_embedding_reindex/retrieval_synthetic_eval_dataset.json...
2024-08-11 09:26:29.161 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:85 - Loading existing synthetic retrieval eval dataset at data/018_finetuned_embedding_reindex/retrieval_synthetic_eval_dataset.json...


In [36]:
retrieval_evaluator.retrieval_eval_dataset.queries

{'faef7066-7118-4800-8f01-0ec95f94a48e': 'What are some restaurants with a lively atmosphere and diverse menus?',
 'd001d2b9-96d2-418c-9a57-6afb98b217df': 'What are some reliable shipping or delivery services that ensure items are received without damage?',
 '2946af06-0e68-47f7-bb5e-60abe195a754': 'What are some affordable parking options in Midtown Nashville?',
 '61eca289-7f41-45d6-b917-d1266401081d': 'What are some businesses or locations that have received negative reviews recently?',
 'b1fa3c41-6c98-4f54-a683-9f5de253c52c': 'What are some ice cream shops that offer a cozy indoor seating experience during winter?',
 '19336816-ba0f-4b4a-ace0-236c1ccce31d': 'What are some popular ice cream shops in Nashville that offer waffle cones?',
 '5bc1432c-2ab6-49f8-8404-9dd71569b22a': 'What are some sushi restaurants that offer a la carte options instead of tasting menus?',
 '2f8e7777-5163-429d-beeb-8b5c958e175a': 'What are some cafes known for their excellent breakfast sandwiches and coffee?',

In [37]:
retrieval_eval_results_df, retrieval_eval_results_full_df = await retrieval_evaluator.aevaluate(cfg, retriever)

Generated queries:
lively atmosphere diverse menus restaurants
"I visited a restaurant with a vibrant ambiance and an extensive menu that offered everything from international cuisine to local favorites. The atmosphere was perfect for a night out with friends!"
Generated queries:
reliable shipping delivery services items received undamaged
"I recently used QuickShip for my deliveries, and I was impressed by their careful handling. My fragile items arrived in perfect condition, and their customer service was very responsive."
Generated queries:
affordable parking options midtown nashville
"I recently visited Midtown Nashville and found several affordable parking spots. One great option was a lot near the main street that charged only $5 for the whole day, making it easy to explore the area without worrying about parking fees."
Generated queries:
negative reviews businesses locations recent
Generated queries:
ice cream shops cozy indoor seating winter
"I visited Sweet Treats last winter 

In [38]:
retrieval_eval_results_df

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,top_50_retrieval_eval,0.7,0.28365,0.014,0.7,0.28365,0.028705


In [39]:
retrieval_eval_results_full_df

,query,expected_ids,retrieved_ids,hit_rate,mrr,precision,recall,ap,ndcg
0,What are some restaurants with a lively atmosp...,[ea2e9ef9-4b90-4206-a74e-b7893a5ff870],"[1414baba-3641-4493-a80b-e55d7b1f60c9, 1bbc15b...",1.0,0.058824,0.02,1.0,0.058824,0.018593
1,What are some reliable shipping or delivery se...,[be0d2e8a-6b8a-4f3e-a8e6-9ed84b10eb12],"[d6d5509c-8b8b-4475-86a5-846cacd0b6fd, 6e15e44...",1.0,0.142857,0.02,1.0,0.142857,0.025844
2,What are some affordable parking options in Mi...,[ffba872c-1151-40d6-ada0-9913eeeddab8],"[91f654a2-d75a-4494-938c-6142cb707f8f, ffba872...",1.0,0.500000,0.02,1.0,0.500000,0.048918
3,What are some businesses or locations that hav...,[711bb479-40d3-40c1-9732-7af7e7a89678],"[15743426-f295-4af1-a255-73393a3d0abc, 39fe191...",0.0,0.000000,0.00,0.0,0.000000,0.000000
4,What are some ice cream shops that offer a coz...,[0fb0975d-58cf-4fff-a13a-aab1d7303b42],"[713a370f-5ad1-4eab-8373-260e662f49b9, 1c8b565...",0.0,0.000000,0.00,0.0,0.000000,0.000000
5,What are some popular ice cream shops in Nashv...,[e3fbd7a0-8a36-4e83-bea5-26d637b418d1],"[e3fbd7a0-8a36-4e83-bea5-26d637b418d1, 23cf94a...",1.0,1.000000,0.02,1.0,1.000000,0.077533
6,What are some sushi restaurants that offer a l...,[fc891a15-7e11-4c45-8c2f-7f6c2761d178],"[1bb2e1be-a1af-4c28-9db0-1140b9433398, fc891a1...",1.0,0.500000,0.02,1.0,0.500000,0.048918
7,What are some cafes known for their excellent ...,[e7061230-bcfc-45d6-8fd2-b67f91d919af],"[72db6c1f-e733-463f-adc2-c0c47279781c, a128cbc...",1.0,0.125000,0.02,1.0,0.125000,0.024459
8,What are some credit card companies known for ...,[b6b645ce-fd9b-4c2d-9b7d-02e11c7d7a0e],"[9da9c08a-d45f-4186-a05b-125879f02e46, 5bd3bf5...",1.0,0.035714,0.02,1.0,0.035714,0.015960
9,What are some restaurants that are known for t...,[788eccdf-896e-46f6-b7ff-07f9094e3252],"[8dab09e3-476d-4c03-855d-6994480cbe92, da0f526...",1.0,0.047619,0.02,1.0,0.047619,0.017386


#### Error Analysis

In [40]:
retrieval_eval_irrelevance_df = (
    retrieval_eval_results_full_df
    .loc[lambda df: df['hit_rate'].lt(1)]
    .sort_values(['hit_rate', 'mrr', 'precision', 'recall', 'ap', 'ndcg'])
)
with pd.option_context('display.max_colwidth', 400):
    display(retrieval_eval_irrelevance_df)

,query,expected_ids,retrieved_ids,hit_rate,mrr,precision,recall,ap,ndcg
3,What are some businesses or locations that have received negative reviews recently?,[711bb479-40d3-40c1-9732-7af7e7a89678],"[15743426-f295-4af1-a255-73393a3d0abc, 39fe191b-f970-43a6-b9be-0052ede1cc31, 8bfe8d23-4c35-446d-b0b2-7f903b905b58, 08e2e3af-ba54-4a74-beb1-0b8d0eda49c4, 269888ad-ae0e-4954-be94-6a0b62d0f9ab, 775076b5-25b3-4660-b733-e09593005368, a38be06b-ab5c-4272-a687-0606fe52ca82, 75450b52-f77e-4a90-8735-05b61346fb82, ffc6fe75-7613-45f1-b05a-4e2a17f2053b, 8fe38ebe-1b4a-4041-809a-e51b2f141148, d81bdcca-c7d9-4...",0.0,0.0,0.0,0.0,0.0,0.0
4,What are some ice cream shops that offer a cozy indoor seating experience during winter?,[0fb0975d-58cf-4fff-a13a-aab1d7303b42],"[713a370f-5ad1-4eab-8373-260e662f49b9, 1c8b565e-dd26-4ec5-9984-40a4aa695935, 7e2ec0e0-101e-4ce1-a0df-4343295e5ac7, 3f5821a8-1a96-440f-ac7a-5a7f2825f806, 9cc0b155-1bfc-47da-8eb2-bc5883ca7caf, dbc2b7b5-f0a8-4145-91a7-c13cf90dd272, 0f1bbcab-c29f-4838-8204-c61771793e19, a5ca356b-295b-4f48-ae93-c1e3706350c9, 91fed185-cbe3-4bdd-9d6e-8dfd8e8962e3, 5b133650-c28b-416c-a69f-56de72051438, 37aa9ada-c94c-4...",0.0,0.0,0.0,0.0,0.0,0.0
11,What are some ice cream shops known for their creamy flavors and homemade waffle cones?,[27309eac-72ce-4940-b4b0-9e449eb1c584],"[ef829e9c-247e-4c36-8901-bea9a686582d, 266a6a36-2d0d-478c-b182-274e8720d784, e81565ef-3fb1-4cd8-9e0a-aeda9be1733a, 44692026-0ae1-4b27-8c88-1d37ca0f788c, c7e43381-4123-42b7-9a25-bea7e0030723, f5461e7e-2345-47e9-b415-755c96ae9b0a, 375fc3d8-e345-430e-a4c8-5ccbf0827cb9, 340d1ba2-e1b2-4392-96bf-882796f80af3, 5cc8ff3b-9a40-4870-abd3-d1c0a181beb0, 6f1dc3f0-b5b6-4e4e-bd88-889ddf4be754, f06eb9e9-8ff2-4...",0.0,0.0,0.0,0.0,0.0,0.0
12,What are some popular shopping destinations for clothing and accessories?,[8c1de488-b6c0-46de-9709-c0b8b0175a91],"[a140641d-2dd0-4cfa-8f2f-800c832d8cff, 2805ede1-1add-4e3a-ae24-0bc5df7b00f1, 36d253bd-508e-4aa4-85a1-ccdf27c71149, f88f65a4-fa11-42ad-a6d1-346134d7e210, 15d198dd-757f-4125-a306-6e94f41b6a3f, 43c32888-7fd2-41b5-b542-a4bbb5db1902, 93130751-4681-4948-a2dc-905b2f2f3685, 43570c4e-4253-4ad3-baa4-2acabe18466c, 802a1475-2e1a-42de-bbc9-2e4f310f5907, 3814e904-d4f4-41a5-8f3b-8d7740e270ab, 8cb31612-629e-4...",0.0,0.0,0.0,0.0,0.0,0.0
16,What are some enjoyable places to visit for leisure activities?,[5fb3b39f-0da6-4207-9ba4-1597008cfa6e],"[e0935b43-ef3f-44b6-81b3-cc73d7298288, dd286dea-b344-4c9f-bb51-93a9e88c1f02, 9d8e47d9-a414-4948-807f-42fdb2285354, e9ddc810-df90-48ce-94cd-0db881d49777, 2977b329-71e1-485c-9fb2-94626f1d9d8b, 54347c0f-7e80-46f8-9139-37e0ca0c74a3, 0e4aba77-e5f6-41bc-bf75-41fc55f50211, 6c1197e5-291a-4928-8bc3-9bb03e698176, a23aa26e-f053-4005-8456-2ee4b6b3a48b, 67c34cbe-130d-4433-ab80-2a245159b6ef, 566a3929-6e73-4...",0.0,0.0,0.0,0.0,0.0,0.0
17,What are some fun activities or attractions that children enjoy?,[625358b5-7d28-44c3-8162-11daaca30089],"[6c1197e5-291a-4928-8bc3-9bb03e698176, e335efa9-f10e-405a-96c2-e90e505aa11a, 2977b329-71e1-485c-9fb2-94626f1d9d8b, 851f2049-ed84-4ccc-aabd-7b2448337e50, e9ddc810-df90-48ce-94cd-0db881d49777, 54347c0f-7e80-46f8-9139-37e0ca0c74a3, 7a19ffbe-0164-46e0-a003-f2dc279fa6ca, e0935b43-ef3f-44b6-81b3-cc73d7298288, dd286dea-b344-4c9f-bb51-93a9e88c1f02, b831c8cf-affe-4346-b679-2f94b6a0653e, 79c3afa3-fefa-4...",0.0,0.0,0.0,0.0,0.0,0.0
21,What are some restaurants known for their excellent customer service and knowledgeable staff?,[5c0c8cdd-ee3b-486b-b573-3b314b60c5f6],"[d22c4631-6b0f-457f-a67a-08847c337a27, c8c734f8-bf69-499d-a22a-4ccc5dda4218, 32f97349-9d5e-45e4-82e7-d8a64d06cf08, 540eb2fe-d2c6-4e6a-a5fb-3eb39edbfe5c, 6cc14d07-5806-44f4-a759-8f69ec35f978, 0e07f374-faf0-4e26-a32b-fd9c3dd65e9d, 88fef5b7-6b95-4eb6-b78b-7b00eeab5362, 08cd7304-54e3-4a12-9fa9-88e83f7e8eeb, 8a005308-22c7-453a-8e97-a93b1251d512, 78b90c9a-8160-4ea6-b88d-e733cbebc3ec, a2d14464-0e20-4...",0.0,0.0,0.0,0.0,0.0,0.0
22,What are some popular tasting rooms or wine bars to visit in the Funk Zone?,[94673875-7bc0-43

In [41]:
for i, row in retrieval_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n============Error #{i+1}=============\n\n")
    print(f"Query:\n{row.query}\n")
    print(f"- Expected contexts:\n")
    expected_nodes = docstore.get_nodes(row.expected_ids)
    for node in expected_nodes:
        print(node.get_content(metadata_mode=MetadataMode.LLM))

    print(f"\n- Retrieved contexts:\n")
    retrieved_nodes = docstore.get_nodes(row.retrieved_ids)
    for i, node in enumerate(retrieved_nodes[:10]):
        print(f"+ Context #{i+1}:\n\n{node.get_content(metadata_mode=MetadataMode.LLM)}\n")



============Error #1=============


Query:
What are some businesses or locations that have received negative reviews recently?

- Expected contexts:

review_stars: 1
biz_name: H&M
biz_address: 827-833 State St
biz_city: Santa Barbara
biz_state: CA
biz_categories: ["Women's Clothing", 'Accessories', "Children's Clothing", "Men's Clothing", 'Adult', 'Shopping', 'Fashion']

Awful.

- Retrieved contexts:

+ Context #1:

review_stars: 1
biz_name: Chase JP Morgan Bank Credit Card Services
biz_address: 
biz_city: Wilmington
biz_state: DE
biz_categories: ['Banks & Credit Unions', 'Financial Services']

That is NOT how you keep clients happy! But judging by all of negative reviews here, I doubt someone would read this or care enough about the reviews to do anything about it. 
You have been warned! Stay away from Chase!

+ Context #2:

review_stars: 1
biz_name: Chase JP Morgan Bank Credit Card Services
biz_address: 
biz_city: Wilmington
biz_state: DE
biz_categories: ['Banks & Credit Unions', '

In [42]:
def check_retrieval_results(query):
    retrieval_results = retriever.retrieve(query)
    for node in retrieval_results:
        display_source_node(node, source_length=1000)
        print(node.get_content(metadata_mode=MetadataMode.EMBED))
        print("\n")
    return retrieval_results

query = "What are some good grocery stores in Nashville that offer curated items and unique local products?"
check_retrieval_results(query)

Generated queries:
grocery stores nashville curated items local products
"I recently visited a grocery store in Nashville that had an amazing selection of curated items and unique local products. The atmosphere was inviting, and I found some great local jams and artisanal cheeses that I couldn't resist!"


**Node ID:** ffba872c-1151-40d6-ada0-9913eeeddab8<br>**Similarity:** 0.06124571945467468<br>**Text:** Interior is gorgeous. Very clean and modern. It'll be great for a lot of Nashville and tourist Instagrammers to get their pose on. 

As far as groceries, lots of curated items from both Nashville and beyond. They have McClures pickles... being from Michigan I got super pumped about those. They also carry Jenis Ice Cream pints which are down right dangerous. Great selection of coffee, sweets, cheeses, charcuterie and wine.

The only reason it's not a full 5 is the parking situation. All the lots in midtown are super expensive or you need change for meters (it's 2017 I don't carry anything that's monetary unless it's a card). If you do meters, it's $1.50 per hour in that area.  Paid lots are about $8 an hour.<br>

review_stars: 4
biz_name: Caviar & Bananas
biz_address: 2031 Broadway
biz_city: Nashville
biz_state: TN
biz_categories: ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners']

Interior is gorgeous. Very clean and modern. It'll be great for a lot of Nashville and tourist Instagrammers to get their pose on. 

As far as groceries, lots of curated items from both Nashville and beyond. They have McClures pickles... being from Michigan I got super pumped about those. They also carry Jenis Ice Cream pints which are down right dangerous. Great selection of coffee, sweets, cheeses, charcuterie and wine.

The only reason it's not a full 5 is the parking situation. All the lots in midtown are super expensive or you need change for meters (it's 2017 I don't carry anything that's monetary unless it's a card). If you do meters, it's $1.50 per hour in that area.  Paid lots ar

**Node ID:** aa27d87a-d02b-41c9-855f-f6ef7151da2b<br>**Similarity:** 0.06069231211546202<br>**Text:** This is a very interesting spot for specialty items and local items like that delicious Nashville fountain of juice. This is a great place to grab coffee, dessert or a salad / sandwich for lunch.<br>

review_stars: 4
biz_name: Caviar & Bananas
biz_address: 2031 Broadway
biz_city: Nashville
biz_state: TN
biz_categories: ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners']

This is a very interesting spot for specialty items and local items like that delicious Nashville fountain of juice. This is a great place to grab coffee, dessert or a salad / sandwich for lunch.




...

**Node ID:** b67a5480-6f42-40c4-8671-3903bbebb631<br>**Similarity:** 0.022462149416728715<br>**Text:** We stop here every time we're in Nashville and miss it when we're gone! They have so many flavors that are all so authentic.<br>

review_stars: 5
biz_name: Mike's Ice Cream
biz_address: 129 2nd Ave N
biz_city: Nashville
biz_state: TN
biz_categories: ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food']

We stop here every time we're in Nashville and miss it when we're gone! They have so many flavors that are all so authentic.




**Node ID:** e8348326-37cf-487a-90de-77bb08ee902d<br>**Similarity:** 0.022254475084663765<br>**Text:** NOPE.  Terrible service - I will not be shopping at that store ever again.  It should not be too much to ask that B&N staffs their stores with people who actually know a few things about the product.  At least a little common courtesy?<br>

review_stars: 1
biz_name: Barnes & Noble Booksellers
biz_address: 8675 River Crossing Blvd
biz_city: Indianapolis
biz_state: IN
biz_categories: ['Shopping', 'Books', 'Mags', 'Music & Video', 'Bookstores']

NOPE.  Terrible service - I will not be shopping at that store ever again.  It should not be too much to ask that B&N staffs their stores with people who actually know a few things about the product.  At least a little common courtesy?




[NodeWithScore(node=TextNode(id_='ffba872c-1151-40d6-ada0-9913eeeddab8', embedding=None, metadata={'review_id': 'VZMjK-Moy-fN08mR2m9JUg', 'user_id': 'U5hfpmnqW65_u10C1i06Dw', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 4, 'funny': 1, 'cool': 0, 'date': '2017-06-23T12:16:51', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'beer_and_wine'", 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': True, 'casual': True}", 'BYOB': None, 'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}", 'BikeParking': 'True', 'BusinessAcc

# Response

In [43]:
from llama_index.core import get_response_synthesizer
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker
from llama_index.core.response_synthesizers.type import ResponseMode

from src.features.append_reference.custom_query_engine import ManualAppendReferenceQueryEngine
from src.features.synthesize_recommendation.custom_tree_summarize import CUSTOM_TREE_SUMMARIZE_PROMPT_SEL

In [44]:
node_postprocessors = []

if cfg.retrieval_cfg.retrieval_similarity_cutoff is not None:
    node_postprocessors.append(SimilarityPostprocessor(similarity_cutoff=cfg.retrieval_cfg.retrieval_similarity_cutoff))

reranker = FlagEmbeddingReranker(model=cfg.retrieval_cfg.rerank_model_name, top_n=cfg.retrieval_cfg.rerank_top_k, use_fp16=True)
node_postprocessors.append(reranker)

response_synthesizer = get_response_synthesizer(
    response_mode=ResponseMode.TREE_SUMMARIZE,
    summary_template=CUSTOM_TREE_SUMMARIZE_PROMPT_SEL
)
query_engine = ManualAppendReferenceQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=node_postprocessors,
)

## Test Query Engine

In [45]:
from llama_index.core.response.notebook_utils import (
    display_source_node,
    display_response,
)

In [46]:
# query = "What are some gastropubs in Indianapolis that are known for their cocktails?"
query = "What are some places to enjoy cold brew coffee"
response = query_engine.query(query)
display_response(response, source_length=1000, show_source=True, show_metadata=True, show_source_metadata=True)

Generated queries:
cold brew coffee places
"I recently visited a cozy café that specializes in cold brew coffee, and I was amazed by their unique flavors and relaxing atmosphere. It's the perfect spot to unwind and enjoy a refreshing drink."


**`Final Response:`** Based on your interest in enjoying cold brew coffee, here are the top recommendations ranked by relevance:

1. **Helena Avenue Bakery**
   - **Location:** 131 Anacapa St, Ste C, Santa Barbara, CA
   - **Review Stars:** 4.5 (average from multiple reviews)
   - **Strengths:** Helena Avenue Bakery is highly praised for its cold brew coffee, which is described as "so good." The bakery offers a cozy atmosphere with a patio for outdoor seating, making it a great spot to relax while enjoying your drink. The menu features a variety of delicious breakfast options and pastries, enhancing the overall experience. The staff is noted for being friendly and accommodating, which adds to the welcoming vibe of the place.
   - **Comparison:** This option stands out for those looking for a full breakfast experience alongside their cold brew, as it offers a diverse menu and a pleasant ambiance.

2. **Mike's Ice Cream**
   - **Location:** 129 2nd Ave N, Nashville, TN
   - **Review Stars:** 5
   - **Strengths:** Mike's Ice Cream is not only known for its ice cream but also offers a variety of coffee options, including cold brew. The establishment is recognized for its friendly staff and quick service, making it a convenient stop for a refreshing drink. The atmosphere is described as chill, perfect for hanging out with friends or enjoying a quiet moment.
   - **Comparison:** While Mike's Ice Cream is primarily an ice cream shop, it provides a solid option for cold brew coffee in a laid-back setting. It may appeal more to those who enjoy a casual environment and might want to indulge in ice cream as well.

3. **Caviar & Bananas**
   - **Location:** 2031 Broadway, Nashville, TN
   - **Review Stars:** 4
   - **Strengths:** Caviar & Bananas offers a diverse menu that includes cold brew coffee, along with a variety of food and wine options. It’s a versatile spot that caters to different needs, whether you're looking for a quick coffee, a meal, or a place to meet up informally. The variety of offerings makes it a unique choice for those who appreciate a more eclectic experience.
   - **Comparison:** This option is ideal for those who want a multi-functional space where they can enjoy coffee along with other amenities like wine and food. However, it may not be as focused on the coffee experience as the other two options.

### Summary
- **Helena Avenue Bakery** is the best choice for a dedicated coffee and breakfast experience, especially for cold brew lovers.
- **Mike's Ice Cream** is perfect for those who want a casual atmosphere and might also enjoy ice cream alongside their coffee.
- **Caviar & Bananas** offers a broader range of options, making it suitable for those looking for a versatile spot but may not be as coffee-centric as the other two.

Choose based on whether you prefer a focused coffee experience, a casual hangout, or a multi-functional venue!


#### Referenced Paragraphs
**Helena Avenue Bakery**

> Pricey ( a ham and cheese croissant was $5.50)  as with anywhere in Santa Barbra but very delicious. Cute space ( a bit hard to find ). The green eggs and ham  breakfast biscuit is to die for.  ($10 I think) The Santa Barbra tri tip sandwich was excellent! Cold brew coffee was good. Lots of good choices in the grab and go section, and our salami and pesto sammy was also great.  (Review ID: <REVIEW_ID>Q3fPo_x6xKxafAzy1hFITg</REVIEW_ID>)

> This review is for the delicious breakfast and coffee. The Green Breakfast Bowl and the Biscuit sandwich were both perfectly cooked, fresh, flavorful. We were really impressed with the quality of the cappuccino and the cold brew coffee. The donut was hearty-tasting and clearly a higher quality of bread compared to your typical donut. We came in at around ten on a Saturday and there were plenty of spots to sit even though it is clearly a popular place to be.  (Review ID: <REVIEW_ID>4sGHeBRdn36yOX3KszhzCA</REVIEW_ID>)

> We got the green eggs and ham, savory scone, cold brew, and mimosas! Their patio is delicious to sit at and enjoy the food! (Review ID: <REVIEW_ID>ybgnv1EGL7U4urhPmPuf4w</REVIEW_ID>)

> Cold brew was sooo good as were the sandwiches we got. Will most definitely be back. (Review ID: <REVIEW_ID>-6iyNp8Vs9kF0IRGXZvsHA</REVIEW_ID>)

> Barista was fast at least. Good latte and cold brew. I should have just ordered a pastry though. (Review ID: <REVIEW_ID>LuLzCVNRkAPjVaRVA__V2A</REVIEW_ID>)

> Got a breakfast sandwich. Biscuit with egg, bacon and cheese. It comes with greens and some tomato spread, i didn't want that. It was delicious. Cold brew decent also. Cashiers were sweet and accommodating. The girl serving the food had a flat affect. Ambiance is a chill, open, industrial vibe. I wish the outdoor space was larger. There is a wine tasting area in the same spot. Would come here again just for that biscuit! (Review ID: <REVIEW_ID>oFtgQKS4xTrvvgL2eTZKHw</REVIEW_ID>)

**Caviar & Bananas**

> Need a bottle of wine? Check!
Need a cold brew coffee with just a splash of almond milk? Check!
Need eggs? Check!
Need a gift basket? Check!
Need a frozen Rose? (Oh didn't know I needed that but I DO!) check! 
Neeeeeed that chocolate? Check!
Need a place for an informal meet up. Check! 
Need a fast bar meal 8:00 pm? Check!

 (Review ID: <REVIEW_ID>TO650uj_N9YlOUFjk9K2eQ</REVIEW_ID>)

**Mike's Ice Cream**

> If it's cold, come grab a coffee instead. They offer all the great styles and flavors of coffee as any other coffee house you'll find. (Review ID: <REVIEW_ID>GHAY9_Kpk-1hcq4Ehfmu1w</REVIEW_ID>)

> I am a big fan of diverse ice cream flavors, and this place definitely has that. If you are cold and not necessarily feeling a frosty treat, they also have different hot chocolate flavors from around the world-the Italian hot chocolate is amazing, by the way! The staff were friendly and willing to go the extra mile to ensure that we were enjoying our coffee.  (Review ID: <REVIEW_ID>S2hL5Cnyzs2ck5xiH8WKGQ</REVIEW_ID>)

> Great place to chill with a drink. Process time is short, and tastes are good.  (Review ID: <REVIEW_ID>t77wB9XveTd1jr1SiK-wkw</REVIEW_ID>)

---

**`Source Node 1/10`**

**Node ID:** 751f7992-a773-439f-8f2e-95a8c6b774d9<br>**Similarity:** -1.8359375<br>**Text:** Pricey ( a ham and cheese croissant was $5.50)  as with anywhere in Santa Barbra but very delicious. Cute space ( a bit hard to find ). The green eggs and ham  breakfast biscuit is to die for.  ($10 I think) The Santa Barbra tri tip sandwich was excellent! Cold brew coffee was good. Lots of good choices in the grab and go section, and our salami and pesto sammy was also great.<br>**Metadata:** {'review_id': 'Q3fPo_x6xKxafAzy1hFITg', 'user_id': 'ha2Lv7WnxvdYnRsqwL1apQ', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-07-17T00:20:41', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'True', 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'HappyHour': 'False', 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'no'"}, 'biz_categories': ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries'], 'biz_hours': {'Friday': '8:0-14:0', 'Monday': '0:0-0:0', 'Saturday': '8:0-14:0', 'Sunday': '8:0-14:0', 'Thursday': '8:0-14:0', 'Tuesday': '8:0-14:0', 'Wednesday': '8:0-14:0'}}<br>

...

**Node ID:** bf65d4b0-9cd7-4eb3-89ab-c3c5e1dcf14b<br>**Similarity:** -4.44140625<br>**Text:** I am a big fan of diverse ice cream flavors, and this place definitely has that. If you are cold and not necessarily feeling a frosty treat, they also have different hot chocolate flavors from around the world-the Italian hot chocolate is amazing, by the way! The staff were friendly and willing to go the extra mile to ensure that we were enjoying our coffee.<br>**Metadata:** {'review_id': 'S2hL5Cnyzs2ck5xiH8WKGQ', 'user_id': 'N7EyLZDccqnLAxw1UVyBNA', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 4, 'useful': 3, 'funny': 0, 'cool': 2, 'date': '2020-01-23T23:56:35', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': 'False', 'Caters': 'False', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': 'False', 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': None, 'HappyHour': None, 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'None', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': None, 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food'], 'biz_hours': {'Friday': '8:0-0:0', 'Monday': '8:0-23:0', 'Saturday': '8:0-0:30', 'Sunday': '8:0-23:0', 'Thursday': '8:0-23:0', 'Tuesday': '8:0-23:0', 'Wednesday': '8:0-23:0'}}<br>

---

**`Source Node 10/10`**

**Node ID:** 693d8c46-3bca-40f7-9245-320e6a92b615<br>**Similarity:** -4.6953125<br>**Text:** Great place to chill with a drink. Process time is short, and tastes are good.<br>**Metadata:** {'review_id': 't77wB9XveTd1jr1SiK-wkw', 'user_id': 'Ph3YHXzuVLTAibVKAWUMAA', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-02-23T22:33:49', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': 'False', 'Caters': 'False', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': 'False', 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': None, 'HappyHour': None, 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'None', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': None, 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food'], 'biz_hours': {'Friday': '8:0-0:0', 'Monday': '8:0-23:0', 'Saturday': '8:0-0:30', 'Sunday': '8:0-23:0', 'Thursday': '8:0-23:0', 'Tuesday': '8:0-23:0', 'Wednesday': '8:0-23:0'}}<br>

{'751f7992-a773-439f-8f2e-95a8c6b774d9': {'review_id': 'Q3fPo_x6xKxafAzy1hFITg',
  'user_id': 'ha2Lv7WnxvdYnRsqwL1apQ',
  'business_id': 'IDtLPgUrqorrpqSLdfMhZQ',
  'review_stars': 4,
  'useful': 0,
  'funny': 0,
  'cool': 0,
  'date': '2018-07-17T00:20:41',
  'biz_name': 'Helena Avenue Bakery',
  'biz_address': '131 Anacapa St, Ste C',
  'biz_city': 'Santa Barbara',
  'biz_state': 'CA',
  'biz_postal_code': '93101',
  'biz_latitude': 34.4144445,
  'biz_longitude': -119.6906718,
  'biz_stars': 4.0,
  'biz_review_count': 389,
  'biz_is_open': 1,
  'biz_attributes': {'AcceptsInsurance': None,
   'Alcohol': "u'none'",
   'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}",
   'BYOB': None,
   'BestNights': None,
   'BikeParking': 'True',
   'BusinessAcceptsBitcoin': None,
   'BusinessAcceptsCreditCards': 'True',
   'BusinessParking': "{'garage': False, 'street': True, 'v

## Response Evaluation

In [47]:
from src.run.eval import ResponseEvaluator

In [48]:
response_evaluator = ResponseEvaluator()

### Synthetic

In [49]:
response_eval_documents, response_synthetic_eval_dataset = response_evaluator.generate_synthetic_dataset(cfg, documents)

2024-08-11 09:27:15.573 | INFO     | src.run.eval.response:generate_synthetic_dataset:40 - Sampling 30 documents for response evaluation...
2024-08-11 09:27:15.580 | INFO     | src.run.eval.response:generate_synthetic_dataset:96 - Loading existing synthetic response eval dataset at data/018_finetuned_embedding_reindex/response_synthetic_eval_dataset.json...


In [50]:
response_synthetic_eval_prediction_dataset = await response_synthetic_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

Batch processing of predictions:   0%|                                                                                                                                                                         | 0/1 [00:00<?, ?it/s]

Generated queries:
highly-rated Japanese restaurants Nashville
"I recently dined at a fantastic Japanese restaurant in Nashville called Koto. The sushi was incredibly fresh, and the ambiance was perfect for a night out. I highly recommend trying their ramen as well!"


Batch processing of predictions:   0%|                                                                                                                                                                         | 0/1 [00:00<?, ?it/s]

Generated queries:
Italian restaurants excellent customer service Reno
"I recently dined at Bella Italia in Reno and was amazed by the outstanding customer service. The staff was attentive and friendly, making the experience truly enjoyable."


Batch processing of predictions:   0%|                                                                                                                                                                         | 0/1 [00:00<?, ?it/s]

...

Batch processing of predictions:   0%|                                                                                                                                                                         | 0/1 [00:00<?, ?it/s]

Generated queries:
breweries craft beers indianapolis
"I recently visited a brewery in Indianapolis that had an impressive selection of craft beers, including IPAs, stouts, and seasonal brews. The atmosphere was lively, and the staff was knowledgeable about their offerings."


Batch processing of predictions:   0%|                                                                                                                                                                         | 0/1 [00:00<?, ?it/s]

Generated queries:
wine bars new orleans atmosphere highly rated
"I visited a fantastic wine bar in New Orleans that had an incredible atmosphere. The ambiance was perfect for a night out, and they offered a great selection of wines."


Batch processing of predictions: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.78s/it]


In [51]:
response_synthetic_mean_scores_df, response_synthetic_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_synthetic_eval_dataset,
    response_synthetic_eval_prediction_dataset,
    dataset_name="synthetic",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [03:06<00:00,  6.20s/it]


In [52]:
response_synthetic_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,4.366667
mean_relevancy_score,0.933333
mean_faithfulness_score,1.000000
mean_context_relevancy_score,0.637500


In [53]:
response_synthetic_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
0,- What are some highly-rated Japanese restaura...,"\nBased on the provided information, the top r...",1.0,5.0,1.0,0.750,"[I had the Tomago (Japanese egg salad, yes the..."
1,- What are some Italian restaurants known for ...,"\nBased on the provided reviews, here are the ...",1.0,3.0,1.0,0.625,"[Service was excellent. , Chicken parm was goo..."
2,- What are some highly-rated wine bars in New ...,"\nBased on the provided information, here are ...",1.0,5.0,1.0,0.750,[A great addition NOLA and the CBD. The wine s...
3,- What are some highly-rated wine bars in New ...,"\nBased on the provided information, here are ...",1.0,4.0,1.0,0.750,[A great addition NOLA and the CBD. The wine s...
4,- What are some ice cream shops that offer uni...,"\nBased on the provided information, here are ...",1.0,4.0,1.0,0.625,[While walking around downtown I kept seeing p...
5,What are some popular coffeehouses in Edmonton...,"\nBased on the provided information, here are ...",1.0,4.0,1.0,0.750,[I went there yesterday afternoon for the firs...
6,What are some highly-rated sports bars in Wesl...,"\nBased on the provided information, the top r...",1.0,5.0,1.0,0.750,[This is the best sports bar and restaurant yo...
7,- What are some cocktail bars in Philadelphia ...,"\nBased on the provided information, here are ...",1.0,5.0,1.0,0.750,[al is great. Atmosphere is fun and friendly a...
8,What are some sports bars in Indianapolis that...,"\nBased on the provided information, here are ...",1.0,4.0,1.0,0.750,[The best part obviously is the scenery and fr...
9,What are some popular wine bars in New Orleans...,"\nBased on the provided information, here are ...",1.0,4.0,1.0,0.750,[YUM!!! Copper vine not only has a great atmos...


#### Error Analysis

In [54]:
response_synthetic_eval_dataset_dict = dict()
for example in response_synthetic_eval_dataset.examples:
    response_synthetic_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [55]:
response_synthetic_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score', 'context_relevancy_score'])

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
23,- What are some pizza places known for their q...,\nBased on the provided reviews for Domino's P...,0.0,3.0,1.0,0.250,[Dominoes is getting better lately about getti...
24,- What are some highly-rated coffee shops in N...,"\nBased on the provided reviews and ratings, h...",0.0,3.0,1.0,0.250,"[The barista was rude, I asked her if the bake..."
1,- What are some Italian restaurants known for ...,"\nBased on the provided reviews, here are the ...",1.0,3.0,1.0,0.625,"[Service was excellent. , Chicken parm was goo..."
18,- What are some sports bars in Indianapolis th...,"\nBased on the provided information, here are ...",1.0,3.0,1.0,0.625,[Atmosphere was like a sports bar. Very clean ...
14,What are some sports bars in Indianapolis that...,"\nBased on the provided information, here are ...",1.0,4.0,1.0,0.125,[Twin peaks has been open for a couple of mont...
25,- What are some mobile phone stores known for ...,\nBased on the provided reviews for T-Mobile a...,1.0,4.0,1.0,0.250,[I get awesome service every time I come here....
10,- What are some ice cream shops in Nashville t...,\nBased on your query for ice cream shops in N...,1.0,4.0,1.0,0.375,[While walking around downtown I kept seeing p...
4,- What are some ice cream shops that offer uni...,"\nBased on the provided information, here are ...",1.0,4.0,1.0,0.625,[While walking around downtown I kept seeing p...
20,- What are some sushi bars in Philadelphia wit...,\nBased on the provided reviews for Tuna Bar i...,1.0,4.0,1.0,0.625,"[Anyways. Food is great, management sucks. The..."
22,- What are some gastropubs in Indianapolis tha...,"\nBased on the provided information, here are ...",1.0,4.0,1.0,0.625,[Def not as good as Chumley's. Probably not co...


In [56]:
CONTEXT_RELEVANCE_SCORE_THRESHOLD = 0.5

response_synthetic_eval_irrelevance_df = (
    response_synthetic_deep_eval_df
    .loc[lambda df: df['context_relevancy_score'].le(CONTEXT_RELEVANCE_SCORE_THRESHOLD)]
    .sort_values(['context_relevancy_score', 'relevancy_score', 'correctness_score', 'faithfulness_score'])
)

with pd.option_context('display.max_colwidth', 400):
    display(response_synthetic_eval_irrelevance_df)

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
14,What are some sports bars in Indianapolis that offer a better dining experience than Twin Peaks?,"\nBased on the provided information, here are some recommendations for sports bars in Indianapolis that may offer a better dining experience than Twin Peaks, ranked by relevance:\n\n### 1. **Hooters**\n - **Strengths**: Hooters is often compared to Twin Peaks, and while some reviews suggest that Twin Peaks is superior, Hooters has a long-standing reputation for its wings and casual dining at...",1.0,4.0,1.0,0.125,[Twin peaks has been open for a couple of months and I had heard good things about the food and the scenery and decided to check it out. I was told by some of my friends that it was substantially better than Hooters in both regards as well. \n\nWe were seated faster than I expected and initially thought it was strange that we were sitting at a very long table along with others. The table was s...
23,"- What are some pizza places known for their quality and flavor in White House, TN?","\nBased on the provided reviews for Domino's Pizza in White House, TN, here are the recommendations ranked by relevance to the user's query about pizza places known for their quality and flavor:\n\n### 1. **Domino's Pizza**\n - **Address:** 3001 Highway 31 W, White House, TN\n - **Review Summary:** \n - **Strengths:** \n - Consistent order accuracy and quick service, as noted in ...",0.0,3.0,1.0,0.250,"[Dominoes is getting better lately about getting out your order pretty quickly while it's still hot. We order about once a week and they have always gotten our order right. , OMG! It was around $26 for an XL pepperoni pizza, nothing else on it. I went to the WH pizza shop on 31 and got one loaded with toppings and it was about the same price and it was delicious! Dominos is horrible & I wil..."
24,- What are some highly-rated coffee shops in Nashville?,"\nBased on the provided reviews and ratings, here are the recommended coffee shops in Nashville, ranked by their relevance to the user's query for highly-rated coffee shops:\n\n### 1. **Caviar & Bananas**\n - **Address:** 2031 Broadway, Nashville, TN\n - **Average Rating:** 3.2 stars (based on multiple reviews)\n - **Strengths:**\n - Offers a variety of options including coffee, brea...",0.0,3.0,1.0,0.250,"[The barista was rude, I asked her if the baked goods were made in house, she smugly responded, "" I don't bake the items, but a baker does.""\n\nThere are limited coffee options in this area of Nashville, especially by our hotel. Unfortunately, the only other option, Starbucks, is the better choice., I stopped by Mike's for coffee, which is the basis of this review. The service was decent as t..."
25,- What are some mobile phone stores known for excellent customer service?,"\nBased on the provided reviews for T-Mobile at 4735 Concord Pike, Wilmington, DE, here are the recommendations ranked by relevance to the user's query about mobile phone stores known for excellent customer service:\n\n### Recommendations:\n\n1. **T-Mobile (4-Star Reviews)**\n - **Strengths**: The store has received multiple 4-star reviews highlighting excellent service, particularly mention...",1.0,4.0,1.0,0.250,"[I get awesome service every time I come here. Thing is, they can get pretty crowded, which means long waits., Okay, so after a couple of trips to this place over the last few years with less than stellar service (one guy was really just sort of gross with a runny nose and no idea whatsoever what he was doing), we have been back twice within the last month and had wonderful, professional serv..."
10,- What are some ice cream shops in Nashville that offer unique flavors?,"\nBased on your query for ice cream shops in Nashville that offer unique flavors, here are the top recommendations, all from Mike's Ice Cream, which is highly praised for its variety and creativity in flavors:\n\n### 1. **Mike's Ice Cream**\n -

In [57]:
for i, row in response_synthetic_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_synthetic_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")



==============Error #1===============


Query:
What are some sports bars in Indianapolis that offer a better dining experience than Twin Peaks?

Context:
Twin peaks has been open for a couple of months and I had heard good things about the food and the scenery and decided to check it out. I was told by some of my friends that it was substantially better than Hooters in both regards as well. 

We were seated faster than I expected and initially thought it was strange that we were sitting at a very long table along with others. The table was separated by a piece of wood with an ax in it. 

Twin Peaks is your typical guys bar, way above Hooters, and on par with tilted kilt.  The thing that puts it over the top is the ICE cold beer served at 29 degrees.

Twin Peaks is your typical guys bar, way above Hooters, and on par with tilted kilt.  The thing that puts it over the top is the ICE cold beer served at 29 degrees.  

If you happen to stumble into Twin Peaks expecting an otherworldly en

In [58]:
# query = "- What are some pizza places known for their quality and flavor in White House, TN?"
query = "What are some sports bars in Indianapolis that offer a better dining experience than Twin Peaks?	"
response = query_engine.query(query)
display_response(response, source_length=1000, show_source=True, show_metadata=True, show_source_metadata=True)

Generated queries:
sports bars dining experience indianapolis better than Twin Peaks
"I recently visited a sports bar in Indianapolis that had an amazing dining experience. The atmosphere was lively, and the food was top-notch, far surpassing what I had at Twin Peaks. I highly recommend checking it out!"


**`Final Response:`** Based on the provided information, here are some recommendations for sports bars in Indianapolis that may offer a better dining experience than Twin Peaks, ranked by relevance:

### 1. **Hooters**
   - **Strengths**: Hooters is often compared to Twin Peaks, and while some reviews suggest that Twin Peaks is superior, Hooters has a long-standing reputation for its wings and casual dining atmosphere. It may offer a more traditional sports bar experience with a focus on classic American fare.
   - **Comparison**: If you're looking for a similar vibe but with a different menu and possibly a more established service model, Hooters could be a good alternative. However, it may not have the same level of customer service or unique atmosphere that some patrons appreciate at Twin Peaks.

### 2. **Tilted Kilt**
   - **Strengths**: Tilted Kilt is another "breastaurant" that is often mentioned alongside Twin Peaks. It offers a similar theme with a focus on sports, food, and attractive waitstaff. The atmosphere is lively, and the menu features a variety of pub-style food.
   - **Comparison**: Tilted Kilt may provide a comparable experience to Twin Peaks but with a different twist on the theme. If you enjoyed the ambiance of Twin Peaks but are looking for a change, Tilted Kilt could be a solid choice.

### 3. **Buffalo Wild Wings**
   - **Strengths**: Known for its extensive menu of wings and a wide selection of beers, Buffalo Wild Wings is a popular sports bar chain that offers a more family-friendly environment. It features numerous TVs for sports viewing and a casual dining experience.
   - **Comparison**: While Buffalo Wild Wings may not have the same themed atmosphere as Twin Peaks, it excels in food variety and is often praised for its wings and sauces. This could be a better option if you're looking for a more traditional sports bar experience with a focus on food quality.

### 4. **The Tap**
   - **Strengths**: The Tap is known for its craft beer selection and a menu that includes burgers, sandwiches, and other American fare. It has a relaxed atmosphere and is often frequented by sports fans.
   - **Comparison**: The Tap may offer a more laid-back dining experience compared to Twin Peaks, with an emphasis on craft beer and quality food. If you're looking for a place to enjoy a game with a good drink and a more relaxed vibe, The Tap could be a great option.

### Summary
- **Hooters** and **Tilted Kilt** provide similar themed experiences to Twin Peaks but may differ in menu and service quality.
- **Buffalo Wild Wings** offers a more traditional sports bar experience with a focus on wings and a family-friendly atmosphere.
- **The Tap** provides a relaxed environment with a strong emphasis on craft beer and quality food.

If your primary goal is to find a sports bar with a better dining experience than Twin Peaks, **Buffalo Wild Wings** and **The Tap** may be your best bets due to their focus on food quality and a more traditional sports bar atmosphere.


#### Referenced Paragraphs
**Twin Peaks**

> Twin peaks has been open for a couple of months and I had heard good things about the food and the scenery and decided to check it out. I was told by some of my friends that it was substantially better than Hooters in both regards as well. 

We were seated faster than I expected and initially thought it was strange that we were sitting at a very long table along with others. The table was separated by a piece of wood with an ax in it.  (Review ID: <REVIEW_ID>l3jFrMXk3h99WlTv-q9gNw</REVIEW_ID>)

> Twin Peaks is your typical guys bar, way above Hooters, and on par with tilted kilt.  The thing that puts it over the top is the ICE cold beer served at 29 degrees. (Review ID: <REVIEW_ID>jAvNoTtB1x2LELl_-7y8kg</REVIEW_ID>)

> Twin Peaks is your typical guys bar, way above Hooters, and on par with tilted kilt.  The thing that puts it over the top is the ICE cold beer served at 29 degrees.   (Review ID: <REVIEW_ID>KsmnyFmghtOx_IbOG9RINg</REVIEW_ID>)

> ...If you happen to stumble into Twin Peaks expecting an otherworldly encounter with David Lynch, you're probably going to be disappointed. 

On the other hand, if you stumble into the brand new Indianapolis location of the Texas-based chain of "breastaurants" called Twin Peaks hoping to enjoy some good eats, good drinks, and scenic views, then you'll likely be incredibly happy. 

Founded in 2005 in Lewisville, Texas, Twin Peaks opened for the dinner hour starting on 12/8/14 at the former Loon Lake Lodge just East of Castleton at the corner of I-69 and East 82nd Street. 

With its servers all wearing the trademark Twin Peaks cleavage and midriff-baring red plaid tops and khaki shorts, the concept behind Twin Peaks may sound familiar but this variation on the familiar theme is far less kitschy and far more personable than other similarly themed restaurants.

The easiest way to describe Twin Peaks is sports bar meets hunting lodge meets bevy of beautiful women. For those who remember Loon L...

> ...Her service was over-the-top, too  (not once did I go lacking for anything), but two other things stood out in my mind as well.  While Twin Peaks is a sports-bar/restaurant, they're very family friendly.  That's unusual for many sports bars/restaurants I've visited. Singles, couples, families, and sports enthusiasts waiting to watch the games were all present. And I can't begin to describe how delicious the food was!  For lunch I had "Mom's Pot Roast," but to be honest mom never made it this great.  My dessert was a Twin Peaks Sundae that was amazing.  For my return trip at dinner, I specifically asked if Arielle was available.  I was informed I was in luck.  She had worked a longer shift than usual today.  I wanted to eat lighter, so I had a cup of Blonde Bean with Bacon Soup (delicious!) and asked Arielle specific questions about the items on the dessert menu I still hadn't tried.  (Again, there was no impatience or indifference.  She was warm and friendly and seemed eager to assist ...

> If I were to describe twin peaks to someone, I would tell them it is an upscale Hooters.  The main difference between Hooters and twin peaks is the quality and the customer service. I'm not saying hooters is bad and does not have good service, but I would pick twin peaks over hooters any day of the week.    (Review ID: <REVIEW_ID>1Tn6LTyYfEwC0TSPsxohXw</REVIEW_ID>)

> I'll NEVER go back to this dump. I'll never recommend this dump. Give me a good experience in a dump and and I'm good. A bad experience in a dump ? NO WAY. 

Get your reality readjusted, Twin Peaks. There are better places for food etc. unless you rate food according to breast exposure. I'll be back in Indy fir business but I'll never be back here. (Review ID: <REVIEW_ID>vne8OJzpDAQ6SmmRQG7nFQ</REVIEW_ID>)

> We stop by at Twin Peaks on our way to Indianapolis to watch the U of L and Purdue game. This was a cool place. It is a sports bar  that had waitresses that are dressed in Specialty little outfits kind of like a checkered shirt with shorts on it kind of fit the outdoor theme of it it was very like Cabela's meets Hooters is the best way I can put it at Giant TVs everywhere the bathrooms were phenomenal they had like TVs right at each year and I really enjoyed that bathroom so huge! And I mean huge just when you thought that a bathroom couldn't be that large it is.  (Review ID: <REVIEW_ID>4U3_8fv3RtU_lif2Ut5ESg</REVIEW_ID>)

> I'm a big sports bar guy and a sports bar food guy and don't expect a sports bar to be 5 star dining. Let me answer a question that people ask me when I said I ate there. "Is it okay to bring your wife and kids there to watch a game?" The servers do wear low cut shirts and shorts, but nothing that was any different than other themed sports bars. When I was there, there were groups of women together and also couples there.  (Review ID: <REVIEW_ID>grTir4NbamCVuxnzfa8sqw</REVIEW_ID>)

> Never even knew this place existed or had heard of it before. Not bad for an average sports bar with sports bar food. The ladies look very nice in their "uniforms" which are very short shorts with their cheeks showing and crop tops with the "peaks" showing hence the name twin peaks! 
        We had mozzarella sticks, chicken wings, and sandwiches with fries. Nothing to write home about but it was good.  (Review ID: <REVIEW_ID>NVYMhlN9utwkEhzthnjmbw</REVIEW_ID>)

---

**`Source Node 1/10`**

**Node ID:** 717e3c37-19b3-4ad7-8afc-e5d3dc288588<br>**Similarity:** 1.0810546875<br>**Text:** Twin peaks has been open for a couple of months and I had heard good things about the food and the scenery and decided to check it out. I was told by some of my friends that it was substantially better than Hooters in both regards as well. 

We were seated faster than I expected and initially thought it was strange that we were sitting at a very long table along with others. The table was separated by a piece of wood with an ax in it.<br>**Metadata:** {'review_id': 'l3jFrMXk3h99WlTv-q9gNw', 'user_id': '-0YrXUvXz8112yHap35V2g', 'business_id': 'seKihQKpGGnCeLuELRQPSQ', 'review_stars': 3, 'useful': 3, 'funny': 1, 'cool': 0, 'date': '2015-01-31T03:20:27', 'biz_name': 'Twin Peaks', 'biz_address': '6880 E 82nd St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46250', 'biz_latitude': 39.9062952, 'biz_longitude': -86.0474634, 'biz_stars': 3.5, 'biz_review_count': 257, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "'full_bar'", 'Ambience': "{u'divey': False, u'hipster': None, u'casual': True, u'touristy': None, u'trendy': None, u'intimate': False, u'romantic': False, u'classy': False, u'upscale': False}", 'BYOB': None, 'BestNights': "{u'monday': False, u'tuesday': True, u'wednesday': False, u'thursday': False, u'friday': True, u'saturday': True, u'sunday': False}", 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'False', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': None, 'latenight': None, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}", 'HappyHour': 'True', 'HasTV': 'True', 'Music': "{'dj': False}", 'NoiseLevel': "'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'False', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'True', 'RestaurantsTakeOut': 'True', 'Smoking': "u'no'", 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Sports Bars', 'American (New)', 'American (Traditional)', 'Nightlife', 'Bars', 'Restaurants'], 'biz_hours': {'Friday': '11:0-19:0', 'Monday': '0:0-0:0', 'Saturday': '11:0-1:0', 'Sunday': '11:0-0:0', 'Thursday': '11:0-22:0', 'Tuesday': '11:0-0:0', 'Wednesday': '11:0-0:0'}}<br>

...

**Node ID:** 6906f71e-fb64-45c4-9cf9-3b10878016e1<br>**Similarity:** -0.59130859375<br>**Text:** I'm a big sports bar guy and a sports bar food guy and don't expect a sports bar to be 5 star dining. Let me answer a question that people ask me when I said I ate there. "Is it okay to bring your wife and kids there to watch a game?" The servers do wear low cut shirts and shorts, but nothing that was any different than other themed sports bars. When I was there, there were groups of women together and also couples there.<br>**Metadata:** {'review_id': 'grTir4NbamCVuxnzfa8sqw', 'user_id': 'sZh1Pp958CUE5Tv3y5xtAQ', 'business_id': 'seKihQKpGGnCeLuELRQPSQ', 'review_stars': 4, 'useful': 3, 'funny': 1, 'cool': 2, 'date': '2015-11-04T16:52:57', 'biz_name': 'Twin Peaks', 'biz_address': '6880 E 82nd St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46250', 'biz_latitude': 39.9062952, 'biz_longitude': -86.0474634, 'biz_stars': 3.5, 'biz_review_count': 257, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "'full_bar'", 'Ambience': "{u'divey': False, u'hipster': None, u'casual': True, u'touristy': None, u'trendy': None, u'intimate': False, u'romantic': False, u'classy': False, u'upscale': False}", 'BYOB': None, 'BestNights': "{u'monday': False, u'tuesday': True, u'wednesday': False, u'thursday': False, u'friday': True, u'saturday': True, u'sunday': False}", 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'False', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': None, 'latenight': None, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}", 'HappyHour': 'True', 'HasTV': 'True', 'Music': "{'dj': False}", 'NoiseLevel': "'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'False', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'True', 'RestaurantsTakeOut': 'True', 'Smoking': "u'no'", 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Sports Bars', 'American (New)', 'American (Traditional)', 'Nightlife', 'Bars', 'Restaurants'], 'biz_hours': {'Friday': '11:0-19:0', 'Monday': '0:0-0:0', 'Saturday': '11:0-1:0', 'Sunday': '11:0-0:0', 'Thursday': '11:0-22:0', 'Tuesday': '11:0-0:0', 'Wednesday': '11:0-0:0'}}<br>

---

**`Source Node 10/10`**

**Node ID:** ef111b5e-9e09-45d3-9271-2209731a9366<br>**Similarity:** -0.64599609375<br>**Text:** Never even knew this place existed or had heard of it before. Not bad for an average sports bar with sports bar food. The ladies look very nice in their "uniforms" which are very short shorts with their cheeks showing and crop tops with the "peaks" showing hence the name twin peaks! 
        We had mozzarella sticks, chicken wings, and sandwiches with fries. Nothing to write home about but it was good.<br>**Metadata:** {'review_id': 'NVYMhlN9utwkEhzthnjmbw', 'user_id': '8dbx6Fy7-ed3tyfo5ZSXXg', 'business_id': 'seKihQKpGGnCeLuELRQPSQ', 'review_stars': 2, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2020-06-14T01:06:22', 'biz_name': 'Twin Peaks', 'biz_address': '6880 E 82nd St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46250', 'biz_latitude': 39.9062952, 'biz_longitude': -86.0474634, 'biz_stars': 3.5, 'biz_review_count': 257, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "'full_bar'", 'Ambience': "{u'divey': False, u'hipster': None, u'casual': True, u'touristy': None, u'trendy': None, u'intimate': False, u'romantic': False, u'classy': False, u'upscale': False}", 'BYOB': None, 'BestNights': "{u'monday': False, u'tuesday': True, u'wednesday': False, u'thursday': False, u'friday': True, u'saturday': True, u'sunday': False}", 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'False', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': None, 'latenight': None, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}", 'HappyHour': 'True', 'HasTV': 'True', 'Music': "{'dj': False}", 'NoiseLevel': "'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'False', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'True', 'RestaurantsTakeOut': 'True', 'Smoking': "u'no'", 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Sports Bars', 'American (New)', 'American (Traditional)', 'Nightlife', 'Bars', 'Restaurants'], 'biz_hours': {'Friday': '11:0-19:0', 'Monday': '0:0-0:0', 'Saturday': '11:0-1:0', 'Sunday': '11:0-0:0', 'Thursday': '11:0-22:0', 'Tuesday': '11:0-0:0', 'Wednesday': '11:0-0:0'}}<br>

{'717e3c37-19b3-4ad7-8afc-e5d3dc288588': {'review_id': 'l3jFrMXk3h99WlTv-q9gNw',
  'user_id': '-0YrXUvXz8112yHap35V2g',
  'business_id': 'seKihQKpGGnCeLuELRQPSQ',
  'review_stars': 3,
  'useful': 3,
  'funny': 1,
  'cool': 0,
  'date': '2015-01-31T03:20:27',
  'biz_name': 'Twin Peaks',
  'biz_address': '6880 E 82nd St',
  'biz_city': 'Indianapolis',
  'biz_state': 'IN',
  'biz_postal_code': '46250',
  'biz_latitude': 39.9062952,
  'biz_longitude': -86.0474634,
  'biz_stars': 3.5,
  'biz_review_count': 257,
  'biz_is_open': 1,
  'biz_attributes': {'AcceptsInsurance': None,
   'Alcohol': "'full_bar'",
   'Ambience': "{u'divey': False, u'hipster': None, u'casual': True, u'touristy': None, u'trendy': None, u'intimate': False, u'romantic': False, u'classy': False, u'upscale': False}",
   'BYOB': None,
   'BestNights': "{u'monday': False, u'tuesday': True, u'wednesday': False, u'thursday': False, u'friday': True, u'saturday': True, u'sunday': False}",
   'BikeParking': 'False',
   'BusinessA

### Manually Curated
Ref: https://docs.llamaindex.ai/en/stable/examples/llama_dataset/ragdataset_submission_template/#1c-creating-a-labelledragdataset-from-scratch-with-manually-constructed-examples

In [59]:
response_curated_eval_dataset = response_evaluator.generate_curated_dataset(cfg)

2024-08-11 09:34:50.934 | INFO     | src.run.eval.response:generate_curated_dataset:125 - Persisting curated response eval dataset at data/025_modify_prompt_with_tree_summarize/response_curated_eval_dataset.json...


In [60]:
response_curated_eval_prediction_dataset = await response_curated_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

Batch processing of predictions:   0%|                                                                                                                                                                         | 0/1 [00:00<?, ?it/s]

Generated queries:
recommended restaurants Santa Barbara nearby
"I recently dined at a fantastic place called The Lark in Santa Barbara. The ambiance was great, and the food was delicious, especially the seasonal dishes. I highly recommend it for anyone looking for a nice meal in the area."


Batch processing of predictions:   0%|                                                                                                                                                                         | 0/1 [00:00<?, ?it/s]

Generated queries:
sports bars dining experience indianapolis better than Twin Peaks
"I visited a great sports bar in Indianapolis last weekend, and it had an amazing atmosphere with delicious food. The menu was diverse, and the service was top-notch, making it a fantastic choice for watching games."


Batch processing of predictions: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:11<00:00, 11.34s/it]


In [61]:
response_curated_mean_scores_df, response_curated_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_curated_eval_dataset,
    response_curated_eval_prediction_dataset,
    dataset_name="curated",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:19<00:00,  9.87s/it]


In [62]:
response_curated_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,3.5000
mean_relevancy_score,1.0000
mean_faithfulness_score,1.0000
mean_context_relevancy_score,0.3125


In [63]:
response_curated_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
0,What are the recommended restaurants nearby Sa...,"\nBased on the provided information, the top r...",1.0,3.0,1.0,0.500,[Finally some good food in downtown Santa Barb...
1,What are some sports bars in Indianapolis that...,\nBased on the provided reviews and context ab...,1.0,4.0,1.0,0.125,[Twin peaks has been open for a couple of mont...


#### Answers for target questions

In [64]:
from IPython.display import Markdown

In [65]:
from src.run.eval.manual_eval_dataset import MANUAL_EVAL_QA

In [66]:
for question, expected_answer in MANUAL_EVAL_QA:
    display(Markdown(f"\n\n### Question: {question}\n"))
    response = query_engine.query(question)
    display_response(response)



### Question: What are the recommended restaurants nearby Santa Barbara?


Generated queries:
recommended restaurants Santa Barbara nearby
"I recently dined at a fantastic place called The Lark in Santa Barbara. The ambiance was perfect, and the food was delicious, especially the seasonal dishes. I highly recommend checking it out if you're in the area!"


**`Final Response:`** Based on the provided information, the top recommendation for restaurants in Santa Barbara is **Helena Avenue Bakery**. Here’s a detailed breakdown of its strengths and how it compares to potential alternatives:

### 1. Helena Avenue Bakery
- **Address**: 131 Anacapa St, Ste C, Santa Barbara, CA
- **Categories**: Food, Restaurants, Salad, Coffee & Tea, Breakfast & Brunch, Sandwiches, Bakeries
- **Review Stars**: 5 (with a few mixed reviews)
  
**Strengths**:
- **Variety of Offerings**: The bakery offers a diverse menu that includes breakfast items, sandwiches, salads, and baked goods, making it suitable for various meal times.
- **Quality of Food**: Many reviews highlight the deliciousness of the food, including their jams, scones, and chicken sandwiches, indicating high-quality ingredients and preparation.
- **Location**: It is conveniently located within walking distance to the beach, making it a great spot for tourists looking to enjoy a meal before or after beach activities.
- **Outdoor Seating**: The availability of outdoor seating, which is regularly disinfected, adds to the dining experience, especially in pleasant weather.
- **Gluten-Free Options**: The bakery is noted for offering gluten-free breakfast options, catering to dietary restrictions.

**Comparison**:
- **For Tourists**: Helena Avenue Bakery is highly recommended for tourists due to its proximity to the beach and the quality of food. It provides a quintessential Santa Barbara experience with a local flavor.
- **For Locals**: While some locals may find better brunch options, the bakery still stands out for its unique offerings and ambiance.

### Summary
Helena Avenue Bakery is the standout recommendation for restaurants in Santa Barbara, particularly for those looking for a casual yet high-quality dining experience. Its strengths lie in its diverse menu, excellent food quality, and convenient location. While there are mixed reviews regarding service, the overall consensus is that the food is exceptional, making it a must-visit for both tourists and locals alike. 

If you are looking for a specific type of cuisine or dining experience beyond what Helena Avenue Bakery offers, further research may be needed to identify additional restaurants in the area. However, based on the provided information, this bakery is the clear choice for a recommended dining experience in Santa Barbara.


#### Referenced Paragraphs
**Helena Avenue Bakery**

> Finally some good food in downtown Santa Barbara.  Everything on the menu is great and could not recommend more.  (Review ID: <REVIEW_ID>8qeD6EhCxUEBj1dO4J4KfA</REVIEW_ID>)

> If you're a tourist and spending the weekend in Santa Barbara, I'd recommend to give this place a go for sure. If you're a local, then not so much.  (Review ID: <REVIEW_ID>Mqh2anPpP_vvLtqCAk_1Xw</REVIEW_ID>)

> Another gem in Santa Barbara! Love the location, walking distance to the beach.  (Review ID: <REVIEW_ID>_S_Lnze72utd6e95FSnw6A</REVIEW_ID>)

> We took a 5 minute walk to the beach boardwalk afterwards. I recommend stopping by! (Review ID: <REVIEW_ID>XwT8iMkd4X548CD_tw99Qg</REVIEW_ID>)

> They offered outdoor seating which they disinfected after each attendee, and their hot chocolate was yummy. Definitely would recommend them, especially if you're in Santa Barbara and looking for a gluten-free breakfast at a cafe. (Review ID: <REVIEW_ID>FwB2dNaeap2oORCc88ubUQ</REVIEW_ID>)

> the service was okay; we had to clean our table ourselves as no one came to clean it. i don't think i'll be returning again given there are better brunch places around downtown santa barbara. (Review ID: <REVIEW_ID>xBgLhU4JvWCL6oHWxXQwjg</REVIEW_ID>)

> O....M....G!!!! I'm so glad this place is in Santa Barbara and not my hometown, because I would never stop eating. Their jams, scones, bagels, breads...delish!!!  (Review ID: <REVIEW_ID>8uAtrjUhdq5nFeHXllueBA</REVIEW_ID>)

> Hands down, the best bakery in Santa Barbara! Great place to grab a quick breakfast in the morning, or a sandwich for lunch while enjoying a glass for wine from The Wine Collective.  (Review ID: <REVIEW_ID>-uxtX-CSXQosWWYdcFgmbg</REVIEW_ID>)

> The chicken sandwich was buttery at the same time it was peppery, and sweet while it was salty. For breakfast on the go, this has got to be one of the best places in Santa Barbara. (Review ID: <REVIEW_ID>OwspRoiswk7AzSIdFK-UDw</REVIEW_ID>)

> I LOVE this spot. Absolutely one of my favorite brunch spots in Santa Barbara. Everything that I've had here has been phenomenal and their food is so well plated!  (Review ID: <REVIEW_ID>Wv8HQMj7at9sELZUKEmIJg</REVIEW_ID>)



### Question: What are some sports bars in Indianapolis that offer a better dining experience than Twin Peaks?


Generated queries:
sports bars dining experience indianapolis better than Twin Peaks
"I visited a fantastic sports bar in Indianapolis that had an amazing atmosphere and delicious food. The wings were incredible, and they had a great selection of beers on tap. It was definitely a step up from Twin Peaks!"


**`Final Response:`** Based on the provided reviews and context about Twin Peaks in Indianapolis, here are some recommendations for sports bars that may offer a better dining experience:

### 1. **Hooters**
   - **Strengths**: Hooters is often compared to Twin Peaks, and while some reviews suggest that Twin Peaks is superior, Hooters has a long-standing reputation for its wings and casual dining atmosphere. It may offer a more traditional sports bar experience with a focus on classic American fare.
   - **Comparison**: If you're looking for a familiar environment with a focus on wings and a casual vibe, Hooters could be a solid choice. However, it may not have the same level of service or unique atmosphere that some patrons appreciate at Twin Peaks.

### 2. **Tilted Kilt**
   - **Strengths**: Similar to Twin Peaks, Tilted Kilt is known for its themed waitstaff and sports bar atmosphere. It often receives praise for its food quality and service, making it a potential step up from Twin Peaks in terms of dining experience.
   - **Comparison**: Tilted Kilt may provide a comparable experience to Twin Peaks but with potentially better food and service. If you enjoy the themed dining experience but want to try something different, this could be a great option.

### 3. **Buffalo Wild Wings**
   - **Strengths**: Known for its extensive menu of wings and a wide variety of sauces, Buffalo Wild Wings is a popular choice for sports fans. The atmosphere is lively, with numerous TVs for game viewing, and it often has a family-friendly environment.
   - **Comparison**: While Buffalo Wild Wings may not have the same themed waitstaff as Twin Peaks, it excels in food variety and a more casual dining experience. This could be a better option if you're looking for a place with a broader menu and a focus on wings.

### 4. **The Sports Page**
   - **Strengths**: This local sports bar is known for its welcoming atmosphere and good food. It often features local sports events and has a loyal customer base. The menu typically includes a variety of American bar food, which may be more satisfying than what some reviewers experienced at Twin Peaks.
   - **Comparison**: The Sports Page may offer a more community-oriented vibe and better food quality compared to Twin Peaks. If you're looking for a local hangout with a friendly atmosphere, this could be a great choice.

### Summary
- **Hooters** and **Tilted Kilt** are both similar to Twin Peaks but may offer better food and service. 
- **Buffalo Wild Wings** provides a more extensive menu and a lively atmosphere, making it a great choice for wing lovers.
- **The Sports Page** offers a local feel with good food, which could be a refreshing alternative to the chain experience of Twin Peaks.

Ultimately, the best choice will depend on your specific preferences for food, atmosphere, and service. If you prioritize a themed experience, Tilted Kilt may be the best option, while Buffalo Wild Wings is ideal for a more casual, wing-focused outing.


#### Referenced Paragraphs
**Twin Peaks**

> Twin peaks has been open for a couple of months and I had heard good things about the food and the scenery and decided to check it out. I was told by some of my friends that it was substantially better than Hooters in both regards as well. 

We were seated faster than I expected and initially thought it was strange that we were sitting at a very long table along with others. The table was separated by a piece of wood with an ax in it.  (Review ID: <REVIEW_ID>l3jFrMXk3h99WlTv-q9gNw</REVIEW_ID>)

> Twin Peaks is your typical guys bar, way above Hooters, and on par with tilted kilt.  The thing that puts it over the top is the ICE cold beer served at 29 degrees. (Review ID: <REVIEW_ID>jAvNoTtB1x2LELl_-7y8kg</REVIEW_ID>)

> Twin Peaks is your typical guys bar, way above Hooters, and on par with tilted kilt.  The thing that puts it over the top is the ICE cold beer served at 29 degrees.   (Review ID: <REVIEW_ID>KsmnyFmghtOx_IbOG9RINg</REVIEW_ID>)

> ...If you happen to stumble into Twin Peaks expecting an otherworldly encounter with David Lynch, you're probably going to be disappointed. 

On the other hand, if you stumble into the brand new Indianapolis location of the Texas-based chain of "breastaurants" called Twin Peaks hoping to enjoy some good eats, good drinks, and scenic views, then you'll likely be incredibly happy. 

Founded in 2005 in Lewisville, Texas, Twin Peaks opened for the dinner hour starting on 12/8/14 at the former Loon Lake Lodge just East of Castleton at the corner of I-69 and East 82nd Street. 

With its servers all wearing the trademark Twin Peaks cleavage and midriff-baring red plaid tops and khaki shorts, the concept behind Twin Peaks may sound familiar but this variation on the familiar theme is far less kitschy and far more personable than other similarly themed restaurants.

The easiest way to describe Twin Peaks is sports bar meets hunting lodge meets bevy of beautiful women. For those who remember Loon L...

> ...Her service was over-the-top, too  (not once did I go lacking for anything), but two other things stood out in my mind as well.  While Twin Peaks is a sports-bar/restaurant, they're very family friendly.  That's unusual for many sports bars/restaurants I've visited. Singles, couples, families, and sports enthusiasts waiting to watch the games were all present. And I can't begin to describe how delicious the food was!  For lunch I had "Mom's Pot Roast," but to be honest mom never made it this great.  My dessert was a Twin Peaks Sundae that was amazing.  For my return trip at dinner, I specifically asked if Arielle was available.  I was informed I was in luck.  She had worked a longer shift than usual today.  I wanted to eat lighter, so I had a cup of Blonde Bean with Bacon Soup (delicious!) and asked Arielle specific questions about the items on the dessert menu I still hadn't tried.  (Again, there was no impatience or indifference.  She was warm and friendly and seemed eager to assist ...

> If I were to describe twin peaks to someone, I would tell them it is an upscale Hooters.  The main difference between Hooters and twin peaks is the quality and the customer service. I'm not saying hooters is bad and does not have good service, but I would pick twin peaks over hooters any day of the week.    (Review ID: <REVIEW_ID>1Tn6LTyYfEwC0TSPsxohXw</REVIEW_ID>)

> I'll NEVER go back to this dump. I'll never recommend this dump. Give me a good experience in a dump and and I'm good. A bad experience in a dump ? NO WAY. 

Get your reality readjusted, Twin Peaks. There are better places for food etc. unless you rate food according to breast exposure. I'll be back in Indy fir business but I'll never be back here. (Review ID: <REVIEW_ID>vne8OJzpDAQ6SmmRQG7nFQ</REVIEW_ID>)

> We stop by at Twin Peaks on our way to Indianapolis to watch the U of L and Purdue game. This was a cool place. It is a sports bar  that had waitresses that are dressed in Specialty little outfits kind of like a checkered shirt with shorts on it kind of fit the outdoor theme of it it was very like Cabela's meets Hooters is the best way I can put it at Giant TVs everywhere the bathrooms were phenomenal they had like TVs right at each year and I really enjoyed that bathroom so huge! And I mean huge just when you thought that a bathroom couldn't be that large it is.  (Review ID: <REVIEW_ID>4U3_8fv3RtU_lif2Ut5ESg</REVIEW_ID>)

> I'm a big sports bar guy and a sports bar food guy and don't expect a sports bar to be 5 star dining. Let me answer a question that people ask me when I said I ate there. "Is it okay to bring your wife and kids there to watch a game?" The servers do wear low cut shirts and shorts, but nothing that was any different than other themed sports bars. When I was there, there were groups of women together and also couples there.  (Review ID: <REVIEW_ID>grTir4NbamCVuxnzfa8sqw</REVIEW_ID>)

> Never even knew this place existed or had heard of it before. Not bad for an average sports bar with sports bar food. The ladies look very nice in their "uniforms" which are very short shorts with their cheeks showing and crop tops with the "peaks" showing hence the name twin peaks! 
        We had mozzarella sticks, chicken wings, and sandwiches with fries. Nothing to write home about but it was good.  (Review ID: <REVIEW_ID>NVYMhlN9utwkEhzthnjmbw</REVIEW_ID>)

### Error Analysis

In [67]:
response_curated_eval_dataset_dict = dict()
for example in response_curated_eval_dataset.examples:
    response_curated_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [68]:
response_curated_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
0,What are the recommended restaurants nearby Sa...,"\nBased on the provided information, the top r...",1.0,3.0,1.0,0.500,[Finally some good food in downtown Santa Barb...
1,What are some sports bars in Indianapolis that...,\nBased on the provided reviews and context ab...,1.0,4.0,1.0,0.125,[Twin peaks has been open for a couple of mont...


In [69]:
response_curated_eval_irrelevance_df = (
    response_curated_deep_eval_df
    .loc[lambda df: df['relevancy_score'].lt(1)]
    .sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])
)

for i, row in response_curated_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_curated_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")

# Chat Bot Quick Test

In [75]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [78]:
query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="reco_review",
        description=(
            "useful for when you want to find places to visit"
            " based on end-user reviews"
        ),
    ),
)

tools = [query_engine_tool]

In [79]:
from llama_index.agent.openai import OpenAIAgent

agent = OpenAIAgent.from_tools(tools, verbose=True)

In [ ]:
response = agent.chat("hi, i am quy")
print(str(response))

response = agent.chat(
    "Where to find some good cold brew coffee?"
)
print(str(response))

response = agent.chat(
    "where is it?"
)
print(str(response))

response = agent.chat(
    "what is the best time to visit?"
)
print(str(response))

response = agent.chat(
    "how about its parking lot options"
)
print(str(response))

response = agent.chat(
    "Ok now I need to find a place for Vietnamese dinner"
)
print(str(response))

# Persist run metadata

In [70]:
from src.run.utils import parse_collect_log, flatten_dict

In [71]:
collect_info = parse_collect_log(collect_fp)

In [72]:
if ARGS.LOG_TO_MLFLOW:
    import mlflow

    logger.info("Logging [COLLECT] info to MLflow...")
    mlflow.log_params(collect_info)
    logger.info("Logging config to MLflow...")
    mlflow.log_params(flatten_dict(cfg.model_dump(), "cfg", sep='.'))
    logger.info(f"Logging Retrieval Synthetic Eval Results to MLflow...")
    retrieval_evaluator.log_to_mlflow(cfg)
    logger.info(f"Logging Response Eval Results to MLflow...")
    try:
        response_evaluator.log_to_mlflow(
            cfg,
            'synthetic',
            response_synthetic_mean_scores_df,
            response_synthetic_deep_eval_df
        )
        response_evaluator.log_to_mlflow(
            cfg,
            'curated',
            response_curated_mean_scores_df,
            response_curated_deep_eval_df
        )
    except:
        ...

2024-08-11 09:35:46.083 | INFO     | __main__:<module>:4 - Logging [COLLECT] info to MLflow...
2024-08-11 09:35:46.107 | INFO     | __main__:<module>:6 - Logging config to MLflow...
/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
2024-08-11 09:35:46.139 | INFO     | __main__:<module>:8 - Logging Retrieval Synthetic Eval Results to MLflow...
2024-08-11 09:35:46.328 | INFO     | __main

# Clean up

In [73]:
if ARGS.LOG_TO_MLFLOW:
    mlflow.end_run()

# Chatbot UI

In [88]:
import gradio as gr

In [89]:
def app_chat(message, history):
    response = agent.chat(message)
    return response

In [94]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(placeholder="<strong>Review Rec Bot</strong><br>I help with recommending places to vist!")
    gr.ChatInterface(
        app_chat,
        chatbot=gr.Chatbot(height=300),
        textbox=gr.Textbox(placeholder="Where do you want to go today?", container=False, scale=7),
        title="Review Rec Bot",
        description="Help users find places to visit based on Yelp reviews",
        theme="soft",
        examples=["Hello", "What are some places to enjoy cold brew coffee"],
        cache_examples=False,
        retry_btn=None,
        undo_btn="Delete Previous",
        clear_btn="Clear",
    )

demo.launch()

Traceback (most recent call last):
  File "/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/nest_asyncio.py", line 26, in run
    loop 

KeyboardInterrupt: 

# Archive